# La Ferme de la Cour - Booking Tracker

In [1]:
# !pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas openpyxl beautifulsoup4

In [30]:
# Enhanced Gmail Booking Tracking System
# This script connects to Gmail, extracts booking information (new, cancelled, modified),
# and saves it to a spreadsheet with proper status tracking

import os
import pandas as pd
import re
import base64
import pickle
import datetime
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import html
import unicodedata
import re
import hashlib
import datetime
import numpy as np

GOAL = 4
    

# Gmail API setup
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# Updated EMAIL_PATTERNS dictionary for all email types
EMAIL_PATTERNS = {
    # Common patterns across email types
    'arrival_date': r'Date d\'Arrivée\s+(\d{2}/\d{2}/\d{4})',
    'departure_date': r'Date de Départ\s+(\d{2}/\d{2}/\d{4})',
    'booking_date': r'booking_date:\s*(\d{2}/\d{2}/\d{4})|Date de la réservation\s+(\d{2}/\d{2}/\d{4})',
    'amount': r'Montant total\s+(\d+[,.]\d+)\s+€',
    'guest_name': r'Nom\s*:\s*(.+?)(?:\n|$)|([A-Z]+\s+[A-Z]+)',
    'phone': r'Téléphone\s*:\s*(.+?)(?:\n|$)|phone:\s*(.+?)(?:">)(.+?)(?:\n|$)',
    'email': r'E-mail\s*:\s*(.+?)(?:\n|$)|email:\s*(.+?)(?:\n|$)',
    'room_type': r'réservation pour\s*:\s*(.+?)(?:\n|$)',
    'reference': r'réservation \| (N°)?([A-Z0-9]+)'
}

def extract_email_type(subject, body):
    """Determine the type of email: Booking, Cancellation, or Modification"""
    subject_lower = subject.lower()
    body_lower = body.lower()
    
    if 'nouvelle réservation' in subject_lower or 'nouvelle réservation' in body_lower:
        return 'Booking'
    elif 'annulation' in subject_lower or 'annulation' in body_lower:
        return 'Cancellation'
    elif 'modification' in subject_lower or 'modification' in body_lower:
        return 'Modification'
    else:
        return 'Unknown'


def extract_reference_number(subject, body):
    """Simplified fallback-based extraction of booking reference number"""

    def clean(text):
        if not text:
            return ""
        text = unicodedata.normalize("NFKD", text)
        text = text.replace("\xa0", " ").replace("\u202f", " ")
        return text.strip()

    full_text = clean(subject) + " " + clean(body)

    # Pattern: one uppercase letter followed by 6+ digits (e.g., U2504074913)
    pattern = r'\b([A-Z][0-9]{6,})\b'
    matches = re.findall(pattern, full_text)

    if matches:
        ref = matches[-1]  # last one usually the most specific
        print(f"✅ Extracted reference: {ref}")
        return ref

    print("⛔ No reference found.")
    return None

def extract_booking_info_html(email_body, booking_source, received_date, email_type, subject):
    """Extract booking information from HTML email using BeautifulSoup"""
    # Start with the email received date as the booking date
    booking_info = {
        'booking_source': booking_source,
        'email_received_date': received_date,
        'booking_date': received_date,  # This is when the booking was actually made
        'email_type': email_type,       # New field to track the type of email
        'status': get_status_from_email_type(email_type)  # Set status based on email type
    }
    
    # Extract reference number
    reference = extract_reference_number(subject, email_body)
    if reference:
        booking_info['reference'] = reference
    
    # Parse with BeautifulSoup
    soup = BeautifulSoup(email_body, 'html.parser')
    
    # Get plain text content
    text_content = soup.get_text()
    
    # Print debug information
    print(f"Extracting {email_type.lower()} information from email...")
    
    # Extract room type (after "réservation pour :")
    room_match = re.search(r'réservation pour\s*:\s*([^\n]+)', text_content)
    if room_match:
        booking_info['room_type'] = room_match.group(1).strip()
        print(f"Room type: {booking_info['room_type']}")
    
    # Extract arrival and departure dates - they appear in a specific format in the email
    arrival_date_match = re.search(r'Date d\'Arrivée\s*(\d{2}/\d{2}/\d{4})', text_content)
    if arrival_date_match:
        booking_info['arrival_date'] = arrival_date_match.group(1).strip()
        print(f"Arrival date: {booking_info['arrival_date']}")
    
    departure_date_match = re.search(r'Date de Départ\s*(\d{2}/\d{2}/\d{4})', text_content)
    if departure_date_match:
        booking_info['departure_date'] = departure_date_match.group(1).strip()
        print(f"Departure date: {booking_info['departure_date']}")
    
    # Extract amount
    amount_match = re.search(r'Montant total\s*(\d+[,.]\d+)\s*€', text_content)
    if amount_match:
        booking_info['amount'] = amount_match.group(1).strip()
        print(f"Amount: {booking_info['amount']} €")
    
    # Extract guest information
    name_match = re.search(r'Nom\s*:\s*([^\n]+)', text_content)
    if name_match and name_match.group(1).strip():
        booking_info['guest_name'] = name_match.group(1).strip()
        print(f"Guest name: {booking_info['guest_name']}")
    else:
        alt_name_match = re.search(r'([A-Z][a-z]+ [A-Z][A-Za-z]+)', text_content)
        if alt_name_match:
            booking_info['guest_name'] = alt_name_match.group(1).strip()
            print(f"Guest name: {booking_info['guest_name']}")
    
    phone_match = re.search(r'Téléphone\s*:\s*([^\n]+)', text_content)
    if phone_match:
        booking_info['phone'] = phone_match.group(1).strip()
        print(f"Phone: {booking_info['phone']}")
    
    email_match = re.search(r'E-mail\s*:\s*([^\n]+)', text_content)
    if email_match:
        booking_info['email'] = email_match.group(1).strip()
        print(f"Email: {booking_info['email']}")
    
    # Add specific fields for modification and cancellation
    if email_type == 'Cancellation':
        booking_info['cancellation_date'] = received_date
        print(f"Cancellation date: {booking_info['cancellation_date']}")
        
        # CRITICAL: For cancellations, manually scan the email log output for missing fields
        # This is a backup mechanism to ensure we capture all the data
        log_text = email_body  # This should contain the printed log info
        
        # Define patterns to extract information from logs
        log_patterns = {
            'room_type': r'Room type: ([^\n]+)',
            'guest_name': r'Guest name: ([^\n]+)',
            'phone': r'Phone: ([^\n]+)',
            'email': r'Email: ([^\n]+)',
            'arrival_date': r'Arrival date: ([^\n]+)',
            'departure_date': r'Departure date: ([^\n]+)',
            'amount': r'Amount: ([^\n€]+)',
            'nights': r'Number of nights: (\d+)'
        }
        
        # Extract from logs any fields that are missing
        for field, pattern in log_patterns.items():
            if field not in booking_info or not booking_info[field]:
                match = re.search(pattern, log_text)
                if match:
                    value = match.group(1).strip()
                    booking_info[field] = value
                    print(f"Recovered {field} from logs: {value}")
        
        # Print final cancellation data
        print("\n== CANCELLATION DATA SUMMARY ==")
        for key, value in booking_info.items():
            print(f"  {key}: {value}")
        print("==============================")
        
    elif email_type == 'Modification':
        booking_info['modification_date'] = received_date
        print(f"Modification date: {booking_info['modification_date']}")
    
    # Calculate number of nights if both dates are available
    if 'arrival_date' in booking_info and 'departure_date' in booking_info:
        try:
            arrival = datetime.datetime.strptime(booking_info['arrival_date'], '%d/%m/%Y')
            departure = datetime.datetime.strptime(booking_info['departure_date'], '%d/%m/%Y')
            nights = (departure - arrival).days
            booking_info['nights'] = nights
            print(f"Number of nights: {nights}")
        except Exception as e:
            booking_info['nights'] = None
            print(f"Error calculating nights: {e}")
    
    print("-" * 40)
    return booking_info

# Add helper functions for date handling
def convert_to_date_format(date_str):
    """Convert various date formats to DD/MM/YYYY"""
    date_str = str(date_str).strip()
    if not date_str or date_str == 'nan' or date_str == 'None':
        return ''
    
    try:
        # Try different date formats
        for fmt in ['%d/%m/%Y', '%Y-%m-%d', '%m/%d/%Y', '%d-%m-%Y', '%d.%m.%Y']:
            try:
                dt = datetime.datetime.strptime(date_str, fmt)
                return dt.strftime('%d/%m/%Y')
            except ValueError:
                continue
                
        # Handle Excel's numeric dates
        try:
            # Excel dates are stored as days since December 30, 1899
            if date_str.replace('.', '', 1).isdigit():
                excel_date = float(date_str)
                dt = datetime.datetime(1899, 12, 30) + datetime.timedelta(days=excel_date)
                return dt.strftime('%d/%m/%Y')
        except:
            pass
            
        # If we get here, could not parse the date
        return date_str
    except Exception as e:
        print(f"Error converting date '{date_str}': {e}")
        return date_str

def convert_date_string_to_datetime(date_str):
    """Convert a date string to a datetime object"""
    if not date_str or date_str == 'nan' or str(date_str).strip() == '':
        return None
        
    try:
        # Try different date formats
        for fmt in ['%d/%m/%Y', '%Y-%m-%d', '%m/%d/%Y', '%d-%m-%Y', '%d.%m.%Y']:
            try:
                return datetime.datetime.strptime(str(date_str).strip(), fmt)
            except ValueError:
                continue
        
        # If we get here, could not parse the date
        return None
    except Exception:
        return None

# Update is_valid_manual_booking to be more lenient
def is_valid_manual_booking(info):
    """Check if manually added booking has the minimum required fields"""
    # For manual bookings, we only need a handful of essential fields
    essential_fields = ['reference', 'guest_name']
    
    for field in essential_fields:
        if field not in info or pd.isna(info[field]) or str(info[field]).strip() == '':
            return False
    
    # At least one of these date fields should be present
    date_fields = ['arrival_date', 'departure_date']
    has_date = False
    for field in date_fields:
        if field in info and not pd.isna(info[field]) and str(info[field]).strip() != '':
            has_date = True
            break
    
    return has_date


# Update validate_booking_info to handle manual bookings more leniently
# def is_valid_booking_info(info, email_type):
#     """Validate booking information based on type"""
#     # If booking source is Manual, use different validation rules
#     if 'booking_source' in info and info['booking_source'] == 'Manual':
#         return is_valid_manual_booking(info)
    
#     def has_value(field):
#         return field in info and not pd.isna(info.get(field)) and str(info.get(field, '')).strip() != ""

#     if email_type == 'Booking':
#         required = ['reference', 'arrival_date', 'departure_date', 'amount', 'guest_name', 'room_type']
#     elif email_type == 'Modification':
#         required = ['reference', 'arrival_date', 'departure_date', 'amount', 'guest_name', 'room_type']
#     elif email_type == 'Cancellation':
#         # Only require reference for cancellations (we will look up rest in Excel)
#         required = ['reference']
#     else:
#         return False

#     for field in required:
#         if not has_value(field):
#             print(f"⛔ Missing or empty required field: {field} → '{info.get(field)}'")

#     return all(has_value(field) for field in required)

def is_valid_booking_info(booking_info, email_type):
    """Check if booking info has the required fields based on email type"""
    if not booking_info or 'reference' not in booking_info:
        return False
    
    # Different validation rules for different email types
    if email_type == 'Booking':
        # For new bookings, require all essential fields
        required_fields = ['guest_name', 'arrival_date', 'departure_date']
    elif email_type == 'Cancellation':
        # For cancellations, we only need the reference number and guest name
        required_fields = ['guest_name']
    elif email_type == 'Modification':
        # For modifications, require reference and some changed field
        required_fields = ['guest_name']
    else:
        # Unknown email type
        return False
    
    # Check if all required fields are present and not empty
    for field in required_fields:
        if field not in booking_info or not booking_info[field]:
            print(f"⚠️ Missing required field for {email_type}: {field}")
            return False
    
    return True


# # Add a utility function to help generate valid references for manual bookings
# def generate_manual_reference(guest_name, date_string):
#     """Generate a reference number for manual bookings"""
#     # Clean the guest name
#     clean_name = ''.join(c for c in guest_name if c.isalnum()).upper()
#     if len(clean_name) < 2:
#         clean_name = 'XX'  # Fallback if name has no letters
    
#     # Get the date components
#     try:
#         booking_date = datetime.datetime.strptime(date_string, '%d/%m/%Y')
#         date_part = booking_date.strftime('%Y%m%d')
#     except:
#         # Fallback to today's date if parsing fails
#         date_part = datetime.datetime.now().strftime('%Y%m%d')
    
#     # Create a hash from name and date
#     combined = f"{clean_name}{date_part}"
#     hash_object = hashlib.md5(combined.encode())
#     hash_hex = hash_object.hexdigest()
    
#     # Format: M (for Manual) + first 2 letters of name + shortened date + 4 chars from hash
#     reference = f"M{clean_name[:2]}{date_part[-4:]}{hash_hex[:4]}".upper()
    
#     return reference

def get_status_from_email_type(email_type):
    """Convert email type to booking status"""
    if email_type == 'Booking':
        return 'Confirmed'
    elif email_type == 'Cancellation':
        return 'Cancelled'
    elif email_type == 'Modification':
        return 'Modified'
    else:
        return 'Unknown'

def authenticate_gmail():
    """Authenticate to Gmail API and return the service"""
    creds = None
    # The file token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # If credentials don't exist or are invalid, let the user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Use a more reliable method for local authentication
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            # Use a specific port and more explicit settings
            creds = flow.run_local_server(
                port=8080,
                prompt='consent',
                authorization_prompt_message="Please authorize this application to access your Gmail account.",
                success_message="Authentication successful! You can close this tab and return to the application."
            )
        
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    service = build('gmail', 'v1', credentials=creds)
    return service

def search_emails(service, query='', max_results=100):
    """Search for emails matching the query"""
    try:
        # Search in either the Booking label or all of inbox
        label_search = "label:Booking" if "Booking" in query else ""
        full_query = f"{query} {label_search}".strip()
        
        result = service.users().messages().list(
            userId='me', q=full_query, maxResults=max_results
        ).execute()
        
        messages = result.get('messages', [])
        return messages
    except Exception as e:
        print(f"An error occurred while searching emails: {e}")
        return []

def get_email_content(service, msg_id):
    """Get the content of a specific email"""
    try:
        message = service.users().messages().get(userId='me', id=msg_id).execute()
        
        # Get email subject and sender
        headers = message['payload']['headers']
        subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'No Subject')
        from_header = next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown Sender')
        
        # Determine booking source
        booking_source = "Website" if "Ferme de la Cour" in from_header else "Booking.com"
        
        # Get email body - improved to handle different message structures
        email_text = ""
        
        if 'parts' in message['payload']:
            for part in message['payload']['parts']:
                if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                    email_text = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8', errors='replace')
                    break
                elif part['mimeType'] == 'text/html' and 'data' in part['body']:
                    # Fallback to HTML if plain text not available
                    html_text = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8', errors='replace')
                    email_text = html_text  # In a real app, you might want to convert HTML to plain text
                    break
        
        # If no parts or still empty, try the body directly
        if not email_text and 'body' in message['payload'] and 'data' in message['payload']['body']:
            email_text = base64.urlsafe_b64decode(message['payload']['body']['data']).decode('utf-8', errors='replace')
        
        return email_text, subject, from_header, booking_source
        
    except Exception as e:
        print(f"An error occurred while getting email content: {e}")
        return "", "", "", ""
    
def identify_repeat_guests(df):
    """
    Identifies repeat guests based on email and phone number,
    adds a 'repeat_guest' column and 'notes' column to the dataframe
    without changing the order of entries.
    
    First visit: repeat_guest = False, visit_count = 1
    Second+ visit: repeat_guest = True, visit_count > 1
    
    Args:
        df: DataFrame containing booking information
    
    Returns:
        DataFrame with added 'repeat_guest' and 'notes' columns in original order
    """
    # Create working copy of dataframe
    result_df = df.copy()
    
    # Store original index for restoring order later
    result_df['original_index'] = range(len(result_df))
    
    # Add notes column if it doesn't exist
    if 'notes' not in result_df.columns:
        result_df['notes'] = ''
    
    # Add repeat_guest column (default False)
    result_df['repeat_guest'] = False
    result_df['visit_count'] = 1  # Default to 1 for all
    
    # Normalize guest information for better matching
    result_df['email_normalized'] = result_df['email'].str.lower().fillna('')
    result_df['phone_normalized'] = result_df['phone'].str.replace(r'[^0-9+]', '', regex=True).fillna('')
    result_df['name_normalized'] = result_df['guest_name'].str.lower().fillna('')
    
    # Create a lookup dictionary to identify unique guests
    guest_dict = {}
    
    # First pass: Build guest identification dictionary
    for idx, row in result_df.iterrows():
        email = row['email_normalized']
        phone = row['phone_normalized']
        name = row['name_normalized']
        
        # Use email as primary identifier, then phone, then name
        if email and len(email) > 3:  # Ensure email is not empty or too short
            key = f"email:{email}"
        elif phone and len(phone) > 5:  # Ensure phone is not empty or too short
            key = f"phone:{phone}"
        elif name and len(name) > 2:  # Ensure name is not empty or too short
            key = f"name:{name}"
        else:
            # Skip if we can't reliably identify the guest
            continue
            
        # Keep track of all indices for this guest
        if key in guest_dict:
            guest_dict[key].append(idx)
        else:
            guest_dict[key] = [idx]
    
    # Second pass: Mark repeat guests and set visit counts
    for guest_key, indices in guest_dict.items():
        # If this guest has multiple bookings
        if len(indices) > 1:
            # Create a temporary list of bookings for this guest
            guest_bookings = []
            for idx in indices:
                if 'booking_date' in result_df.columns:
                    try:
                        booking_date = pd.to_datetime(result_df.at[idx, 'booking_date'], 
                                                      format='%d/%m/%Y', errors='coerce')
                    except:
                        # If parsing fails, use a very old date as fallback
                        booking_date = pd.Timestamp('1900-01-01')
                else:
                    # If no booking date, use index as proxy for chronological order
                    booking_date = pd.Timestamp('1900-01-01')
                
                guest_bookings.append((idx, booking_date))
            
            # Sort by booking date
            guest_bookings.sort(key=lambda x: x[1])
            
            # First booking is not a repeat
            first_idx = guest_bookings[0][0]
            result_df.at[first_idx, 'repeat_guest'] = False
            result_df.at[first_idx, 'visit_count'] = 1
            
            # Mark subsequent bookings as repeats with incrementing visit counts
            for i, (idx, _) in enumerate(guest_bookings[1:], 2):
                result_df.at[idx, 'repeat_guest'] = True
                result_df.at[idx, 'visit_count'] = i
    
    # Clean up temporary columns
    result_df = result_df.drop(['email_normalized', 'phone_normalized', 'name_normalized'], axis=1)
    
    # Restore original order
    result_df = result_df.sort_values('original_index')
    result_df = result_df.drop('original_index', axis=1)
    
    # Count how many repeat bookings we found
    repeat_bookings = result_df['repeat_guest'].sum()
    
    print(f"✅ Identified {repeat_bookings} bookings from repeat guests")
    
    return result_df

def process_all_booking_emails(search_terms=['nouvelle réservation', 'annulation', 'modification'], 
                               max_emails=100, debug=False, weekly_booking_goal=GOAL):
    """Main function to process all types of booking emails and manual bookings"""
    service = authenticate_gmail()
    all_bookings = []
    
    # First, check for existing file and load any manual bookings
    manual_bookings = []
    existing_file = 'lafermedelacour_bookings.xlsx'
    existing_df = None
    
    if os.path.exists(existing_file):
        try:
            existing_df = pd.read_excel(existing_file)
            print(f"Found existing bookings file with {len(existing_df)} records.")
            
            # Extract manual bookings (identified by 'Manual' in booking_source column)
            if 'booking_source' in existing_df.columns:
                manual_entries = existing_df[existing_df['booking_source'] == 'Manual'].copy()
                if not manual_entries.empty:
                    print(f"Found {len(manual_entries)} manual bookings in the Excel file.")
                    
                    # Fix for manual entries: ensure all required fields have proper values
                    for col in manual_entries.columns:
                        # Fill empty email-related fields with default values for manual bookings
                        if col == 'email_received_date' and manual_entries[col].isna().any():
                            manual_entries[col] = manual_entries['booking_date'].fillna(datetime.datetime.now().strftime('%d/%m/%Y'))
                        elif col == 'email_type' and manual_entries[col].isna().any():
                            manual_entries[col] = 'Booking'
                        elif col == 'booking_date' and manual_entries[col].isna().any():
                            manual_entries[col] = datetime.datetime.now().strftime('%d/%m/%Y')
                    
                    # Convert to list of dictionaries for processing
                    manual_bookings = manual_entries.to_dict('records')
        except Exception as e:
            print(f"Error processing existing file: {e}")
            print("Will try to continue by creating a new DataFrame for manual entries...")
            
            # Try to recover manual entries even if there was an error
            try:
                if existing_df is not None and 'booking_source' in existing_df.columns:
                    manual_entries = existing_df[existing_df['booking_source'] == 'Manual'].copy()
                    if not manual_entries.empty:
                        # Create a minimal dataset with only the required fields
                        minimal_entries = []
                        for _, row in manual_entries.iterrows():
                            minimal_entry = {
                                'booking_source': 'Manual',
                                'reference': row.get('reference', 'UNKNOWN'),
                                'status': row.get('status', 'Confirmed'),
                                'guest_name': row.get('guest_name', ''),
                                'arrival_date': row.get('arrival_date', ''),
                                'departure_date': row.get('departure_date', ''),
                                'amount': row.get('amount', ''),
                                'room_type': row.get('room_type', ''),
                                'phone': row.get('phone', ''),
                                'email': row.get('email', ''),
                                'nights': row.get('nights', ''),
                                'email_received_date': datetime.datetime.now().strftime('%d/%m/%Y'),
                                'booking_date': datetime.datetime.now().strftime('%d/%m/%Y'),
                                'email_type': 'Booking'
                            }
                            minimal_entries.append(minimal_entry)
                        manual_bookings = minimal_entries
            except Exception as nested_e:
                print(f"Could not recover manual entries: {nested_e}")
    
    # Process each search term to get email-based bookings
    for search_term in search_terms:
        print(f"\nProcessing emails with term: '{search_term}'...")
        
        messages = search_emails(service, search_term, max_emails)
        
        for message in messages:
            msg_id = message['id']
            message_detail = service.users().messages().get(userId='me', id=msg_id).execute()
            internal_date = int(message_detail['internalDate']) / 1000  # Convert to seconds
            received_date = datetime.datetime.fromtimestamp(internal_date).strftime('%d/%m/%Y')
            
            email_body, subject, from_header, booking_source = get_email_content(service, msg_id)
            
            # Determine email type
            email_type = extract_email_type(subject, email_body)
            
            if email_type != 'Unknown':
                # Debug output omitted for brevity...

                # Extract booking information based on email type
                booking_info = extract_booking_info_html(email_body, booking_source, received_date, email_type, subject)

                # CRITICAL: Print what we have right after extraction
                if email_type == 'Cancellation':
                    print("\n====== EXTRACTION DATA CHECK ======")
                    print(f"Cancellation email for reference: {booking_info.get('reference', 'UNKNOWN')}")
                    print("Extracted data:")
                    for key, value in booking_info.items():
                        print(f"  {key}: {value}")
                    print("===================================")

                # IMPORTANT: For cancellations, verify we have the necessary fields
                if email_type == 'Cancellation':
                    # Add this critical fix:
                    print("DEBUG: Ensuring cancellation data integrity...")

                    # Make sure these fields are available and properly converted
                    critical_fields = ['reference', 'guest_name', 'room_type', 'phone', 'email', 
                                      'arrival_date', 'departure_date', 'amount', 'nights']

                    # Loop through the logs to find extracted data
                    for field in critical_fields:
                        if field not in booking_info or not booking_info[field]:
                            # Try to find this field in the extracted data logs
                            if field == 'room_type':
                                match = re.search(r'Room type: (.+)', email_body)
                                if match:
                                    booking_info[field] = match.group(1).strip()
                                    print(f"  Recovered {field}: {booking_info[field]}")

                            elif field == 'guest_name':
                                match = re.search(r'Guest name: (.+)', email_body)
                                if match:
                                    booking_info[field] = match.group(1).strip()
                                    print(f"  Recovered {field}: {booking_info[field]}")

                            elif field == 'phone':
                                match = re.search(r'Phone: (.+)', email_body)
                                if match:
                                    booking_info[field] = match.group(1).strip()
                                    print(f"  Recovered {field}: {booking_info[field]}")

                            elif field == 'email':
                                match = re.search(r'Email: (.+)', email_body)
                                if match:
                                    booking_info[field] = match.group(1).strip()
                                    print(f"  Recovered {field}: {booking_info[field]}")

                            # Add similar patterns for other fields

                    # Print booking_info after possible recovery
                    print("\n🔍 DEBUG: Cancellation data after recovery:")
                    for key, value in booking_info.items():
                        print(f"  {key}: {value}")

                # Add this critical fix - create a copy of the data to prevent modifications
                # Before adding to all_bookings
                if is_valid_booking_info(booking_info, email_type):
                    if email_type == 'Cancellation':
                        print("\n====== BEFORE ADDING TO ALL_BOOKINGS ======")
                        print(f"Cancellation email for reference: {booking_info.get('reference', 'UNKNOWN')}")
                        for key, value in booking_info.items():
                            print(f"  {key}: {value}")
                        print("===========================================")

                    all_bookings.append(booking_info)
                else:
                    print(f"⛔ Skipped invalid {email_type} email. Subject: {subject}")

    
    # Add manual bookings to the list of all bookings
    if manual_bookings:
        print(f"Adding {len(manual_bookings)} manual bookings to processing...")
        all_bookings.extend(manual_bookings)

    # Process and save the bookings
    if all_bookings:
        # Convert to DataFrame
        df = pd.DataFrame(all_bookings)
        
        manual_mask = df['booking_source'] == 'Manual'
        if manual_mask.any():
            print("\nStandardizing date formats for manual bookings...")
            date_columns = ['arrival_date', 'departure_date', 'booking_date', 'cancellation_date', 'modification_date']

            for column in date_columns:
                if column in df.columns:
                    # Convert datetime objects to DD/MM/YYYY string format
                    df.loc[manual_mask, column] = df.loc[manual_mask, column].apply(
                        lambda x: x.strftime('%d/%m/%Y') if isinstance(x, datetime.datetime) else x
                    )
            print("✅ Date formats standardized for manual bookings")
        
        # Prepare new_email_df early for safe downstream access
        new_email_df = df[df['booking_source'] != 'Manual'].copy()
        new_email_df = new_email_df.sort_values(by='email_received_date')
        new_email_df = new_email_df.drop_duplicates(subset='reference', keep='last')


        # Check what happened to cancellations during conversion
        if 'email_type' in df.columns:
            cancellations = df[df['email_type'] == 'Cancellation']
            if not cancellations.empty:
                print("\n====== CANCELLATIONS IN DATAFRAME ======")
                for _, row in cancellations.iterrows():
                    print(f"Cancellation reference: {row.get('reference', 'UNKNOWN')}")
                    for col in ['guest_name', 'room_type', 'phone', 'email']:
                        print(f"  {col}: {row[col] if col in row.index else 'NOT FOUND'}")
                print("=========================================")
        
        # Make sure date fields are strings for consistent handling
        date_columns = ['arrival_date', 'departure_date', 'booking_date', 'email_received_date', 
                        'cancellation_date', 'modification_date']
        
        for col in date_columns:
            if col in df.columns:
                # Ensure dates are strings in DD/MM/YYYY format
                df[col] = df[col].astype(str)
                df[col] = df[col].apply(lambda x: convert_to_date_format(x) if x and x.strip() != 'nan' else '')
        
        # For manual bookings, set default values for required columns
        manual_mask = df['booking_source'] == 'Manual'
        if 'email_type' in df.columns and manual_mask.any():
            df.loc[manual_mask & df['email_type'].isna(), 'email_type'] = 'Booking'
        
        if 'email_received_date' in df.columns and manual_mask.any():
            today = datetime.datetime.now().strftime('%d/%m/%Y')
            df.loc[manual_mask & df['email_received_date'].isna(), 'email_received_date'] = today
        
        if 'booking_date' in df.columns and manual_mask.any():
            today = datetime.datetime.now().strftime('%d/%m/%Y')
            df.loc[manual_mask & df['booking_date'].isna(), 'booking_date'] = today
        
        # Calculate nights for any entry missing it
        if 'arrival_date' in df.columns and 'departure_date' in df.columns:
            for idx, row in df.iterrows():
                if pd.isna(row.get('nights')) or str(row.get('nights')).strip() == '':
                    try:
                        arrival = convert_date_string_to_datetime(row['arrival_date'])
                        departure = convert_date_string_to_datetime(row['departure_date'])
                        if arrival and departure:
                            nights = (departure - arrival).days
                            df.at[idx, 'nights'] = nights
                    except Exception as e:
                        print(f"Could not calculate nights for row {idx}: {e}")
        
        # If existing Excel file exists, merge with new updates
        if existing_df is not None:
            try:
                # Create a copy without manual entries (we already have them in all_bookings)
                email_entries = existing_df[existing_df['booking_source'] != 'Manual'].copy()
                
                # First, deduplicate based on reference to keep only the last email
                # Deduplicate new emails
#                 if not new_email_df.empty:
#                     new_email_df = new_email_df.sort_values(by='email_received_date').drop_duplicates(subset='reference', keep='last')

                # ✅ Cache booking/modification data BEFORE calling process_booking_updates
                latest_booking_data = {
                    b['reference']: b
                    for b in all_bookings
                    if b['email_type'] in ['Booking', 'Modification'] and 'reference' in b
                }

                # Merge updates
                merged_email_df = process_booking_updates(email_entries, new_email_df, latest_booking_data)


                # Now combine merged email data with manual entries
                manual_df = df[df['booking_source'] == 'Manual'].copy()

                # Convert any integer columns in merged_email_df to float to prevent NaN errors
                for col in merged_email_df.columns:
                    if merged_email_df[col].dtype in ['int64', 'int32']:
                        merged_email_df[col] = merged_email_df[col].astype(float)

                    # For any numeric column that might contain comma-separated values
                    if col in ['amount', 'nights']:
                        merged_email_df[col] = merged_email_df[col].apply(
                            lambda x: float(str(x).replace(',', '.')) if isinstance(x, str) else x
                        )

                # Add this right after creating manual_df
                print("\nDEBUG: Converting all integer columns to float in manual_df")
                for col in manual_df.columns:
                    if manual_df[col].dtype in ['int64', 'int32']:
                        print(f"  Converting manual_df[{col}] from {manual_df[col].dtype} to float")
                        manual_df[col] = manual_df[col].astype(float)

                    # For any numeric column that might contain comma-separated values
                    if col in ['amount', 'nights']:
                        manual_df[col] = manual_df[col].apply(
                            lambda x: float(str(x).replace(',', '.')) if isinstance(x, str) else x
                        )
                
                print("\nDEBUG: Column types before concat:")
                for col in merged_email_df.columns:
                    print(f"  merged_email_df[{col}]: {merged_email_df[col].dtype}")

                for col in manual_df.columns:
                    print(f"  manual_df[{col}]: {manual_df[col].dtype}")

                # Check overlap of columns
                shared_cols = set(merged_email_df.columns).intersection(set(manual_df.columns))
                print(f"Shared columns: {shared_cols}")

                df = pd.concat([merged_email_df, manual_df], ignore_index=True)
                
                if 'room_type' in df.columns:
                    print("\nStandardizing room names...")
                    df['room_type'] = df['room_type'].apply(standardize_room_name)
                    print("✅ Room names standardized")
    
            except Exception as e:
                print(f"Error merging with existing file: {e}")
                print("Creating new file with all entries instead.")

                # Try more aggressive error handling for amount fields
                try:
                    # Attempt to fix amount field in the full dataframe
                    if 'amount' in df.columns:
                        df['amount'] = df['amount'].apply(
                            lambda x: float(str(x).replace(',', '.')) if isinstance(x, str) else x
                        )
                except Exception as nested_e:
                    print(f"Could not fix amount field: {nested_e}")
        
        # Sort by email received date
        if 'email_received_date' in df.columns:
            try:
                df['email_received_date'] = pd.to_datetime(df['email_received_date'], 
                                                          format='%d/%m/%Y', 
                                                          errors='coerce')
                df = df.sort_values('email_received_date', ascending=True)
                df['email_received_date'] = df['email_received_date'].dt.strftime('%d/%m/%Y')
            except Exception as e:
                print(f"Error sorting by email_received_date: {e}")
                # Try an alternative sorting if possible
                try:
                    if 'arrival_date' in df.columns:
                        df['temp_arrival'] = pd.to_datetime(df['arrival_date'], 
                                                           format='%d/%m/%Y', 
                                                           errors='coerce')
                        df = df.sort_values('temp_arrival', ascending=False)
                        df = df.drop('temp_arrival', axis=1)
                except:
                    print("Could not sort by alternative date")
            
        # Debug line
        print("\n📊 Total unique references:", df['reference'].nunique())
        
        # Identify repeat guests
        df = identify_repeat_guests(df)
        
        # Debug output to see what's happening with manual bookings
        print("\nDEBUG - Manual bookings columns and types:")
        if manual_bookings:
            sample_booking = manual_bookings[0]
            for key, value in sample_booking.items():
                print(f"{key}: {value} (type: {type(value)})")

        # When processing amounts, add this:
        # Convert comma-based decimals to period-based
        if 'amount' in df.columns:
            print("\nConverting amount formats...")
            df['amount'] = df['amount'].astype(str).str.replace(',', '.').replace('', np.nan)
            try:
                df['amount'] = df['amount'].astype(float)
                print("Converted amount to float successfully.")
            except Exception as e:
                print(f"Error converting amount: {e}")

        # Save to Excel
        # First, drop the email_received_date column as it's a duplicate of booking_date
        if 'email_received_date' in df.columns:
            print("\nDropping email_received_date column (duplicate of booking_date)")
            df = df.drop('email_received_date', axis=1)

        df.to_excel('lafermedelacour_bookings.xlsx', index=False)
        print("✅ Saved to Excel. Final row count:", len(df))
        
        # Generate enhanced analytics
        try:
            analytics = weekly_booking_analytics(df, weekly_booking_goal)
            print("\nBooking Analytics:")
            print(f"Total Bookings: {analytics['total_bookings']}")
            
            # Display status breakdown
            if 'status_counts' in analytics:
                print("\nBooking Status Breakdown:")
                for status, count in analytics['status_counts'].items():
                    print(f"  {status}: {count} bookings")
            
            # Display goal achievement stats
            if 'goal_achievement_rate' in analytics:
                print(f"\nWeekly Booking Goal: {weekly_booking_goal} bookings/week")
                print(f"Goal Achievement Rate: {analytics['goal_achievement_rate']:.1%}")
            
            # Display other analytics data
            if isinstance(analytics['average_stay'], (int, float)):
                print(f"Average Stay: {analytics['average_stay']:.1f} nights")
            
            if isinstance(analytics['total_revenue'], (int, float)):
                print(f"Total Revenue (Confirmed Bookings): €{analytics['total_revenue']:.2f}")
            
            if 'high_occupancy_days_count' in analytics:
                print(f"Days with Multiple Rooms Booked: {analytics['high_occupancy_days_count']}")
            
            if 'repeat_guest_count' in analytics:
                print(f"Repeat Guests: {analytics['repeat_guest_count']}")
                print(f"Additional Bookings from Repeat Guests: {analytics['repeat_bookings_count']}")
            
            # Display weekly booking information
            print("\nWeekly Booking Counts (Goal: {weekly_booking_goal}/week):")
            if isinstance(analytics['weekly_bookings'], pd.Series):
                for week, count in analytics['weekly_bookings'].items():
                    goal_status = "✓" if count >= weekly_booking_goal else "✗"
                    print(f"{week}: {count} bookings {goal_status}")
            
            # Create enhanced visualizations
            try:
                create_booking_visualizations(df, weekly_booking_goal)
                print("\nEnhanced booking visualizations saved as 'booking_analytics.png'")
            except Exception as e:
                print(f"Could not create visualizations: {e}")
            
            # Create room occupancy calendar
            try:
                create_occupancy_calendar(df)
            except Exception as e:
                print(f"Could not create occupancy calendar: {e}")
            
            # Analyze cancellations and modifications
            try:
                cancellation_analysis = analyze_cancellations_and_modifications(df)
                print("\nCancellation and Modification Analysis:")
                print(f"Cancellation Rate: {cancellation_analysis['cancellation_rate']:.1%}")
                print(f"Modification Rate: {cancellation_analysis['modification_rate']:.1%}")
                if 'avg_days_before_arrival_cancelled' in cancellation_analysis:
                    print(f"Average Days Before Arrival When Cancelled: {cancellation_analysis['avg_days_before_arrival_cancelled']:.1f}")
            except Exception as e:
                print(f"Could not analyze cancellations: {e}")
            
        except Exception as e:
            print(f"\nError generating analytics AAAAAAAAAAA: {e}")
        
        return df
    else:
        print("No booking emails found.")
        return pd.DataFrame(all_bookings) if all_bookings else pd.DataFrame()

def process_booking_updates(existing_df, new_df, latest_booking_data=None):
    """
    Process booking updates with clear priority rules:
    1. Start with existing data
    2. For each reference number, apply modifications and cancellations
    3. Preserve manual bookings during processing
    4. Ensure only ONE row per reference number in final result
    """
    print('********************************************************************')
    print('DEBUG: Starting process_booking_updates')
    
    if latest_booking_data is None:
        latest_booking_data = {}
    
    # If no existing data, just return the new data
    if existing_df.empty:
        print('DEBUG: No existing data, returning new data')
        return new_df.copy()
    
    # Working copies to avoid modifying originals
    result_df = existing_df.copy()
    new_entries = new_df.copy()
    
    # IMPORTANT: Keep a complete copy of the original data for retrieving missing fields later
    original_df = existing_df.copy()
    
    # Convert comma decimal separators in numeric fields
    for df in [result_df, new_entries]:
        for col in ['amount', 'nights']:
            if col in df.columns:
                df[col] = df[col].apply(
                    lambda x: float(str(x).replace(',', '.')) if isinstance(x, str) else x
                )
    
    # Convert integer columns to float so they can handle NaN values
    for col in result_df.columns:
        if result_df[col].dtype == 'int64' or result_df[col].dtype == 'int32':
            result_df[col] = result_df[col].astype(float)
            
    # CRITICAL FIX: Also convert boolean columns to object type, which can handle NaN values
    for col in result_df.columns:
        if result_df[col].dtype == 'bool':
            print(f"  Converting boolean column {col} to object type")
            result_df[col] = result_df[col].astype(object)
            
    # Also convert integer columns in new_entries to float
    for col in new_entries.columns:
        if new_entries[col].dtype == 'int64' or new_entries[col].dtype == 'int32':
            new_entries[col] = new_entries[col].astype(float)

    # Add more comprehensive debug information
    print(f"DEBUG: Column dtypes in result_df after conversion:")
    for col in result_df.columns:
        print(f"  {col}: {result_df[col].dtype}")

    print(f"DEBUG: Column dtypes in new_entries after conversion:")
    for col in new_entries.columns:
        print(f"  {col}: {new_entries[col].dtype}")
    
    # Tracking counters
    updated_count = 0
    cancelled_count = 0
    added_count = 0
    
    # DEBUG: Print column names for debugging
    print(f"DEBUG: Existing DataFrame columns: {result_df.columns.tolist()}")
    print(f"DEBUG: New DataFrame columns: {new_entries.columns.tolist()}")
    
    # Step 1: Create a dictionary of existing references for quick lookup
    existing_refs = {}
    for idx, row in result_df.iterrows():
        ref = str(row['reference'])  # Convert to string for consistency
        existing_refs[ref] = idx
    
    # DEBUG: Print existing references
    print(f"DEBUG: Found {len(existing_refs)} existing references")
    
    # Step 2: Process new entries (updates and new bookings)
    for _, new_row in new_entries.iterrows():
        ref = str(new_row['reference'])  # Convert to string for consistency
        email_type = new_row['email_type']
        booking_source = new_row.get('booking_source', '')
        
        print(f"\nDEBUG: Processing {email_type} for reference {ref}")
        
        # Check if this reference already exists
        if ref in existing_refs:
            idx = existing_refs[ref]
            existing_source = result_df.at[idx, 'booking_source'] if 'booking_source' in result_df.columns else ''
            
            # Special handling for manual bookings:
            # If existing entry is manual and new entry is from email, 
            # preserve manual entry unless it's being cancelled
            if existing_source == 'Manual' and booking_source != 'Manual' and email_type != 'Cancellation':
                print(f"⚠️ Preserving manual booking with reference: {ref}")
                continue
            
            # CASE 1: MODIFICATION - Update booking details but preserve contact info
            if email_type == 'Modification':
                # First, save contact info if it exists in original booking
                preserved_fields = {}
                for field in ['guest_name', 'phone', 'email']:
                    if field in result_df.columns and not pd.isna(result_df.at[idx, field]) and result_df.at[idx, field] != '':
                        preserved_fields[field] = result_df.at[idx, field]

                print(f"DEBUG: Modification - preserved fields: {preserved_fields}")

                # Add debugging for assignment operations
                print(f"DEBUG: Examining assignment operations for modification ref {ref}")
                for col in new_row.index:
                    if col in result_df.columns:
                        if pd.isna(new_row[col]) and result_df[col].dtype in ['int64', 'int32']:
                            print(f"  ⚠️ Warning: Trying to assign NaN to integer column {col}!")
                            # Convert to float first
                            print(f"  Converting column {col} from {result_df[col].dtype} to float")
                            result_df[col] = result_df[col].astype(float)
                        result_df.at[idx, col] = new_row[col]

                # Restore contact info that was missing in modification but present in original
                for field, value in preserved_fields.items():
                    if pd.isna(result_df.at[idx, field]) or result_df.at[idx, field] == '':
                        result_df.at[idx, field] = value

                updated_count += 1
                print(f"✅ Updated booking reference: {ref}")



            # CASE 2: CANCELLATION
            elif email_type == 'Cancellation':
                print('DEBUG: Processing cancellation')

                # Skip cancellation for manual bookings if needed
                if 'booking_source' in result_df.columns and result_df.at[idx, 'booking_source'] == 'Manual':
                    print(f"⚠️ Cannot cancel manual booking with reference: {ref}. Skipping.")
                    continue

                # CRITICAL FIX: First, check the cancellation email for complete data
                print(f"\n🔍 DEBUG: New cancellation row data inspection:")

                # Updated: Only preserve contact fields, not booking details
                contact_fields = ['room_type', 'guest_name', 'phone', 'email']
                has_complete_data = True

                for field in contact_fields:
                    if field in new_row.index:
                        value = new_row[field]
                        print(f"  {field}: {value} (type: {type(value)}, is_null: {pd.isna(value) if hasattr(value, 'isna') else 'N/A'})")
                        if pd.isna(value) or str(value).strip() in ['', 'nan']:
                            has_complete_data = False

                print(f"  Has complete contact data: {has_complete_data}")

                # Check if the booking is already cancelled
                already_cancelled = False
                if 'status' in result_df.columns and result_df.at[idx, 'status'] == 'Cancelled':
                    already_cancelled = True
                    print(f"⚠️ Booking with reference {ref} is already cancelled. Updating with any new info.")

                # CRITICAL FIX: Before proceeding, preserve existing contact info
                existing_contact_info = {}
                for field in contact_fields:
                    if field in result_df.columns and not pd.isna(result_df.at[idx, field]) and result_df.at[idx, field] != '':
                        existing_contact_info[field] = result_df.at[idx, field]
                        print(f"  Preserving existing {field}: {existing_contact_info[field]}")

                # Update status fields
                result_df.at[idx, 'status'] = 'Cancelled'
                result_df.at[idx, 'email_type'] = 'Cancellation'

                # Add cancellation date
                if 'cancellation_date' in new_row and not pd.isna(new_row['cancellation_date']):
                    result_df.at[idx, 'cancellation_date'] = new_row['cancellation_date']
                elif 'email_received_date' in new_row and not pd.isna(new_row['email_received_date']):
                    result_df.at[idx, 'cancellation_date'] = new_row['email_received_date']

                # MODIFICATION: Clear booking details while preserving contact info
                # Clear booking details
                for field in ['amount', 'arrival_date', 'departure_date', 'nights']:
                    if field in result_df.columns:
                        result_df.at[idx, field] = None
                        print(f"  Cleared booking detail: {field}")

                # CRITICAL FIX: Comprehensive approach to preserving ONLY contact data
                field_data = {}

                # 1. First, look in the current cancellation email
                for field in contact_fields:
                    if field in new_row and pd.notna(new_row[field]) and str(new_row[field]).strip() not in ['', 'nan']:
                        field_data[field] = new_row[field]
                        print(f"  Found {field} = {new_row[field]} in current cancellation email")

                # 2. Next, look in historical entries if we need more data
                historical_entries = original_df[
                                                (original_df['reference'].astype(str) == ref) &
                                                (original_df['status'].isin(['Confirmed', 'Modified']))
                                            ]

                if not historical_entries.empty:
                    for field in contact_fields:
                        if field not in field_data or pd.isna(field_data[field]) or str(field_data[field]).strip() in ['', 'nan']:
                            # Find non-empty values
                            valid_values = historical_entries[~historical_entries[field].isna() & 
                                                            (historical_entries[field].astype(str).str.strip() != '') &
                                                            (historical_entries[field].astype(str).str.strip() != 'nan')]

                            if not valid_values.empty:
                                field_data[field] = valid_values.iloc[-1][field]
                                print(f"  Found {field} = {field_data[field]} in historical data")

                # 3. Use the existing data as last resort
                for field in contact_fields:
                    if field not in field_data or pd.isna(field_data[field]) or str(field_data[field]).strip() in ['', 'nan']:
                        if field in existing_contact_info:
                            field_data[field] = existing_contact_info[field]
                            print(f"  Using existing {field} = {field_data[field]}")

                # 4. Use cached recent booking data (from this session)
                if ref in latest_booking_data:
                    recent = latest_booking_data[ref]
                    for field in contact_fields:
                        if field not in field_data or str(field_data[field]).strip() in ['', 'nan']:
                            if recent.get(field):
                                field_data[field] = recent[field]
                                print(f"  Pulled {field} from cached booking data: {recent[field]}")
                                
                # Add debugging for assignment operations
                print(f"DEBUG: Examining assignment operations for modification ref {ref}")
                for col in new_row.index:
                    if col in result_df.columns:
                        if pd.isna(new_row[col]) and result_df[col].dtype in ['int64', 'int32']:
                            print(f"  ⚠️ Warning: Trying to assign NaN to integer column {col}!")
                            # Convert to float first
                            print(f"  Converting column {col} from {result_df[col].dtype} to float")
                            result_df[col] = result_df[col].astype(float)
                        result_df.at[idx, col] = new_row[col]

                # 5. Apply all the data we found, even if it's coming from different sources
                print(f"\n📝 Final data for cancelled booking {ref}:")
                for field, value in field_data.items():
                    result_df.at[idx, field] = value
                    print(f"  {field}: {value}")

                cancelled_count += 1
                # Don't double-count cancellations
                if already_cancelled:
                    cancelled_count -= 1
                    print(f"ℹ️ Updated existing cancellation for reference: {ref}")
                else:
                    print(f"🚫 Cancelled booking reference: {ref}")

            # CASE 3: NEW BOOKING for existing reference
            elif email_type == 'Booking':
                print('DEBUG: Processing new booking for existing reference')

                # Similar to modification but with different logging
                preserved_fields = {}
                for field in ['guest_name', 'phone', 'email']:
                    if field in result_df.columns and not pd.isna(result_df.at[idx, field]) and result_df.at[idx, field] != '':
                        preserved_fields[field] = result_df.at[idx, field]

                print(f"DEBUG: New booking - preserved fields: {preserved_fields}")

                # Debug what's happening 
                print(f"DEBUG: Examining assignment operations for reference {ref}")
                for col in new_row.index:
                    if col in result_df.columns:
#                         print(f"  Column {col}: result_df type = {result_df[col].dtype}, new value type = {type(new_row[col])}, value = {new_row[col]}")

                        # Check for incompatible assignments
                        if pd.isna(new_row[col]):
                            if result_df[col].dtype in ['int64', 'int32']:
                                print(f"  ⚠️ Warning: Trying to assign NaN to integer column {col}!")
                                result_df[col] = result_df[col].astype(float)
                            elif result_df[col].dtype == 'bool':
                                print(f"  ⚠️ Warning: Trying to assign NaN to boolean column {col}!")
                                result_df[col] = result_df[col].astype(object)

                # Now perform the assignments
                for col in new_row.index:
                    if col in result_df.columns:
                        try:
                            result_df.at[idx, col] = new_row[col]
                        except Exception as e:
                            print(f"  ❌ Error assigning to column {col}: {e}")
                            # Try to fix and retry
                            if pd.isna(new_row[col]):
                                if result_df[col].dtype == 'bool':
                                    result_df[col] = result_df[col].astype(object)
                                elif result_df[col].dtype in ['int64', 'int32']:
                                    result_df[col] = result_df[col].astype(float)
                                # Try again
                                result_df.at[idx, col] = new_row[col]

                # Restore preserved fields
                for field, value in preserved_fields.items():
                    if pd.isna(result_df.at[idx, field]) or result_df.at[idx, field] == '':
                        result_df.at[idx, field] = value

                print(f"⚠️ Replaced existing booking reference: {ref}")

        # New booking reference - add to results
        else:
            print(f"DEBUG: Adding new booking reference: {ref}")
            
            # Create a new row in the result dataframe
            new_idx = len(result_df)
            for col in new_row.index:
                if col in result_df.columns:
                    result_df.at[new_idx, col] = new_row[col]
            
            # Update our reference dictionary
            existing_refs[ref] = new_idx
            
            added_count += 1
            print(f"➕ Added new booking reference: {ref}")
    
    # Final verification step - make sure we have exactly one row per reference
    result_df = result_df.drop_duplicates(subset=['reference'], keep='last')
    
    print(f"\n📊 Summary:")
    print(f"  - Added {added_count} new bookings")
    print(f"  - Updated {updated_count} existing bookings")
    print(f"  - Cancelled {cancelled_count} bookings")
    print(f"  - Final unique booking count: {len(result_df)}")
    
    # Before returning the result, make sure we preserve any notes from existing data
    if 'notes' in existing_df.columns:
        for idx, row in result_df.iterrows():
            ref = str(row['reference'])
            # Find this reference in the original data
            original_matches = existing_df[existing_df['reference'].astype(str) == ref]
            
            if not original_matches.empty:
                # Get the notes from the original entry
                original_notes = original_matches.iloc[0].get('notes', '')
                
                # Only copy if we have notes and the current entry doesn't
                if not pd.isna(original_notes) and original_notes != '':
                    if 'notes' not in result_df.columns:
                        result_df['notes'] = ''
                    result_df.at[idx, 'notes'] = original_notes
    
    # Make sure notes column exists
    if 'notes' not in result_df.columns:
        result_df['notes'] = ''
    
    # DEBUG: Final check of cancelled bookings to ensure contact info is preserved
    cancelled_refs = result_df[result_df['status'] == 'Cancelled']['reference'].tolist()
    print(f"\nDEBUG: Final check of {len(cancelled_refs)} cancelled bookings:")
    for ref in cancelled_refs:
        row_idx = result_df[result_df['reference'] == ref].index[0]
        print(f"Cancelled booking {ref}:")
        for field in ['room_type', 'guest_name', 'phone', 'email']:
            if field in result_df.columns:
                value = result_df.at[row_idx, field]
                print(f"  {field}: {value}")
    
    return result_df

def weekly_booking_analytics(df, weekly_booking_goal=GOAL):
    """Generate weekly booking analytics with European date format ranges and goal tracking"""
    if df.empty:
        return "No data available for analytics"
    
    analytics_result = {
        'total_bookings': len(df),
        'weekly_booking_goal': weekly_booking_goal
    }
    
    # Add status breakdown
    if 'status' in df.columns:
        status_counts = df['status'].value_counts().to_dict()
        analytics_result['status_counts'] = status_counts
    
    # Process booking dates (when bookings were made)
    if 'booking_date' in df.columns:
        try:
            # Convert to datetime
            df['booking_date'] = pd.to_datetime(df['booking_date'], format='%d/%m/%Y', errors='coerce')
            df_booking = df.dropna(subset=['booking_date']).copy()
            
            # For analytics purposes, only consider confirmed bookings
            if 'status' in df_booking.columns:
                df_booking = df_booking[df_booking['status'].isin(['Confirmed', 'Modified'])]
            
            if not df_booking.empty:
                # Create a new column for the Monday of the week
                df_booking.loc[:, 'week_start'] = df_booking['booking_date'].apply(
                    lambda x: x - pd.Timedelta(days=x.weekday())  # weekday() returns 0 for Monday
                )
                
                # Count bookings by week
                weekly_counts = df_booking.groupby('week_start').size()
                
                # Convert index to European date range format (Monday-Sunday)
                date_ranges = {}
                goal_achievement = {}
                
                for week_start in weekly_counts.index:
                    # Sunday is 6 days after Monday
                    week_end = week_start + pd.Timedelta(days=6)
                    date_range_key = f"{week_start.strftime('%d/%m/%Y')} - {week_end.strftime('%d/%m/%Y')}"
                    count = int(weekly_counts[week_start])
                    date_ranges[date_range_key] = count
                    
                    # Track if the booking goal was met
                    goal_achievement[date_range_key] = count >= weekly_booking_goal
                
                analytics_result['weekly_bookings'] = pd.Series(date_ranges)
                analytics_result['goal_achievement'] = pd.Series(goal_achievement)
                analytics_result['goal_achievement_rate'] = sum(goal_achievement.values()) / len(goal_achievement) if goal_achievement else 0
            else:
                analytics_result['weekly_bookings'] = "No valid booking dates found"
        except Exception as e:
            print(f"Error processing booking dates: {e}")
            analytics_result['weekly_bookings'] = f"Error processing booking dates: {e}"
    else:
        analytics_result['weekly_bookings'] = "Booking date information not available"
    
    # Calculate average stay if nights column exists (only for confirmed/modified bookings)
    if 'nights' in df.columns:
        try:
            # Filter for confirmed/modified bookings if status exists
            if 'status' in df.columns:
                df_active = df[df['status'].isin(['Confirmed', 'Modified'])]
            else:
                df_active = df
                
            nights = pd.to_numeric(df_active['nights'], errors='coerce')
            analytics_result['average_stay'] = nights.mean()
        except Exception as e:
            print(f"Error calculating average stay: {e}")
            analytics_result['average_stay'] = f"Error calculating average stay: {e}"
    else:
        analytics_result['average_stay'] = "Stay duration information not available"
    
    # Calculate total revenue if amount column exists (only for confirmed/modified bookings)
    if 'amount' in df.columns:
        try:
            # Filter for confirmed/modified bookings if status exists
            if 'status' in df.columns:
                df_active = df[df['status'].isin(['Confirmed', 'Modified'])]
            else:
                df_active = df
                
            # Clean and convert amount values
            amounts = df_active['amount'].astype(str).str.replace(',', '.').replace('', '0')
            analytics_result['total_revenue'] = pd.to_numeric(amounts, errors='coerce').sum()
        except Exception as e:
            print(f"Error calculating total revenue: {e}")
            analytics_result['total_revenue'] = f"Error calculating total revenue: {e}"
    else:
        analytics_result['total_revenue'] = "Revenue information not available"
    
    # Add upcoming arrivals analysis
    if 'arrival_date' in df.columns:
        try:
            # Convert to datetime if not already
            df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%d/%m/%Y', errors='coerce')
            
            # Filter for valid dates and confirmed/modified bookings
            df_arrivals = df.dropna(subset=['arrival_date']).copy()
            if 'status' in df_arrivals.columns:
                df_arrivals = df_arrivals[df_arrivals['status'].isin(['Confirmed', 'Modified'])]
            
            # Only include future arrivals
            today = pd.Timestamp.now().normalize()
            future_arrivals = df_arrivals[df_arrivals['arrival_date'] >= today]
            
            if not future_arrivals.empty:
                # Group by week
                future_arrivals.loc[:, 'arrival_week_start'] = future_arrivals['arrival_date'].apply(
                    lambda x: x - pd.Timedelta(days=x.weekday())
                )
                
                # Count arrivals by week
                weekly_arrivals = future_arrivals.groupby('arrival_week_start').size()
                
                # Convert to date range format
                arrival_ranges = {}
                for week_start in weekly_arrivals.index:
                    week_end = week_start + pd.Timedelta(days=6)
                    date_range_key = f"{week_start.strftime('%d/%m/%Y')} - {week_end.strftime('%d/%m/%Y')}"
                    arrival_ranges[date_range_key] = int(weekly_arrivals[week_start])
                
                analytics_result['upcoming_arrivals'] = pd.Series(arrival_ranges)
            else:
                analytics_result['upcoming_arrivals'] = "No upcoming arrivals found"
        except Exception as e:
            print(f"Error calculating upcoming arrivals: {e}")
            analytics_result['upcoming_arrivals'] = f"Error calculating upcoming arrivals: {e}"
    else:
        analytics_result['upcoming_arrivals'] = "Arrival date information not available"
    
    return analytics_result

def analyze_cancellations_and_modifications(df):
    """Analyze cancellation and modification patterns"""
    result = {}
    
    if 'email_type' not in df.columns:
        return {'error': 'No email type information available'}
    
    # Calculate basic rates
    total_bookings = len(df)
    cancellations = df[df['email_type'] == 'Cancellation'].copy()
    modifications = df[df['email_type'] == 'Modification'].copy()

    result['cancellation_count'] = len(cancellations)
    result['modification_count'] = len(modifications)
    result['cancellation_rate'] = len(cancellations) / total_bookings if total_bookings > 0 else 0
    result['modification_rate'] = len(modifications) / total_bookings if total_bookings > 0 else 0
    
    # Analyze how long before arrival cancellations happen
    if not cancellations.empty and 'arrival_date' in cancellations.columns and 'cancellation_date' in cancellations.columns:
        try:
            # Convert dates to datetime
            cancellations['arrival_date'] = pd.to_datetime(cancellations['arrival_date'], format='%d/%m/%Y', errors='coerce')
            cancellations['cancellation_date'] = pd.to_datetime(cancellations['cancellation_date'], format='%d/%m/%Y', errors='coerce')
            
            # Calculate days between cancellation and arrival
            cancellations['days_before_arrival'] = (cancellations['arrival_date'] - cancellations['cancellation_date']).dt.days
            
            result['avg_days_before_arrival_cancelled'] = cancellations['days_before_arrival'].mean()
            result['min_days_before_arrival_cancelled'] = cancellations['days_before_arrival'].min()
            result['max_days_before_arrival_cancelled'] = cancellations['days_before_arrival'].max()
            
            # Analyze cancellations by timeframe
            timeframes = {
                'last_minute': cancellations[cancellations['days_before_arrival'] <= 7],
                'short_notice': cancellations[(cancellations['days_before_arrival'] > 7) & (cancellations['days_before_arrival'] <= 30)],
                'advance': cancellations[cancellations['days_before_arrival'] > 30]
            }
            
            for timeframe, data in timeframes.items():
                result[f'{timeframe}_cancellations'] = len(data)
                result[f'{timeframe}_cancellation_rate'] = len(data) / len(cancellations) if len(cancellations) > 0 else 0
            
        except Exception as e:
            print(f"Error analyzing cancellation timing: {e}")
    
    # Analyze modifications
    if not modifications.empty and 'modification_date' in modifications.columns:
        try:
            # Get most commonly modified rooms
            if 'room_type' in modifications.columns:
                result['most_modified_rooms'] = modifications['room_type'].value_counts().to_dict()
            
            # Analyze when modifications happen
            if 'arrival_date' in modifications.columns:
                modifications['arrival_date'] = pd.to_datetime(modifications['arrival_date'], format='%d/%m/%Y', errors='coerce')
                modifications['modification_date'] = pd.to_datetime(modifications['modification_date'], format='%d/%m/%Y', errors='coerce')
                
                modifications['days_before_arrival'] = (modifications['arrival_date'] - modifications['modification_date']).dt.days
                
                result['avg_days_before_arrival_modified'] = modifications['days_before_arrival'].mean()
        except Exception as e:
            print(f"Error analyzing modifications: {e}")
    
    return result

def analyze_guest_nationalities(df):
    """Create a visualization of guest nationalities based on phone numbers"""
    if 'phone' not in df.columns:
        print("Cannot analyze nationalities: missing 'phone' column")
        return
    
    # Function to extract country from phone number
    def extract_country(phone):
        # Convert to string if it's not already a string
        if not isinstance(phone, str):
            phone = str(phone)
        
        # Handle empty phone numbers
        if not phone or pd.isna(phone):
            return "Unknown"
        
        # Strip all spaces
        phone = phone.strip().replace(' ', '')
        
        # Define country code mappings
        country_codes = {
            '32': 'Belgium',
            '33': 'France',
            '49': 'Germany',
            '31': 'Netherlands',
            '352': 'Luxembourg',
            '44': 'United Kingdom',
            '39': 'Italy',
            '34': 'Spain',
            '351': 'Portugal',
            '41': 'Switzerland',
            '36': 'Hungary'
        }
        
        # Remove leading '+' if present
        if phone.startswith('+'):
            phone = phone[1:]
            
            # Special case for Belgian local numbers with +0 prefix
            if phone.startswith('0') and len(phone) >= 10:
                Belgian_mobile_prefixes = ['04', '05', '047', '048', '049']
                if any(phone.startswith(prefix) for prefix in Belgian_mobile_prefixes):
                    return 'Belgium'
        
        # Remove leading '00' (international prefix) if present
        if phone.startswith('00'):
            phone = phone[2:]
        
        # Special case for Belgian local numbers with 0 prefix
        if phone.startswith('0') and len(phone) >= 9:
            Belgian_mobile_prefixes = ['04', '05', '047', '048', '049']
            if any(phone.startswith(prefix) for prefix in Belgian_mobile_prefixes):
                return 'Belgium'
            
        # Direct check for country codes at the start of the string
        for code in sorted(country_codes.keys(), key=len, reverse=True):
            if phone.startswith(code):
                return country_codes[code]
        
        # If we reach here, the phone number format is not recognized
        return "Unknown"
    
    # Extract nationalities
    df['guest_nationality'] = df['phone'].apply(extract_country)
    
    # For debugging: Print sample of phone number to nationality mappings
    print("\nSample phone number to nationality mappings:")
    for idx, row in df.sample(min(10, len(df))).iterrows():
        print(f"Phone: {row['phone']} → Nationality: {row['guest_nationality']}")
    
    # Count nationalities
    nationality_counts = df['guest_nationality'].value_counts()
    
    # Create a pie chart for nationalities
    plt.figure(figsize=(10, 8))
    
    # Color palette - use more distinct colors
    colors = plt.cm.tab20(np.linspace(0, 1, len(nationality_counts)))
    
    # Create the pie chart
    wedges, texts, autotexts = plt.pie(
        nationality_counts,
        labels=nationality_counts.index,
        autopct='%1.1f%%',
        colors=colors,
        shadow=False,
        startangle=90,
        wedgeprops={'edgecolor': 'w', 'linewidth': 1}
    )
    
    # Enhance font size of labels
    plt.setp(texts, size=12)
    plt.setp(autotexts, size=10, weight='bold')
    
    plt.title('Guest Nationalities Distribution', fontsize=16, fontweight='bold')
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    
    # Add count numbers to the legend
    legend_labels = [f"{nat} ({count})" for nat, count in zip(nationality_counts.index, nationality_counts.values)]
    plt.legend(wedges, legend_labels, title="Nationalities", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    
    plt.tight_layout()
    plt.savefig('guest_nationalities.png', dpi=300)
    plt.close()
    print("Guest nationalities visualization saved as 'guest_nationalities.png'")
    
    # Return the data for further analysis if needed
    return nationality_counts

# Modify the room distribution visualization to use consistent colors
def create_room_distribution_with_consistent_colors(df):
    """Create room type distribution with consistent colors matching the calendar view"""
    if 'room_type' not in df.columns:
        print("Cannot create room distribution: missing 'room_type' column")
        return
    
    # Standardize room names
    df_rooms = df.copy()
    df_rooms['room_type'] = df_rooms['room_type'].apply(standardize_room_name)
    
    # Define room colors (same as in create_monthly_calendar)
    room_colors = {
        'Laurasie de la Cour': 'pink',
        'Tibert de la Cour': '#FFBF00',
        'Odette de la Cour': 'grey',
        'Léon de la Cour': 'teal'
    }
    
    # Count room types
    room_counts = df_rooms['room_type'].value_counts()
    
    # Prepare colors list in the same order as room_counts
    colors = [room_colors.get(room, 'blue') for room in room_counts.index]
    
    plt.figure(figsize=(10, 6))
    
    # Create the bar chart with consistent colors
    bars = plt.bar(range(len(room_counts)), room_counts.values, color=colors)
    
    # Set x-axis labels
    plt.xticks(range(len(room_counts)), room_counts.index, rotation=45, ha='right')
    
    # Calculate percentages
    total = room_counts.sum()
    percentages = [(count / total) * 100 for count in room_counts.values]
    
    # Add value and percentage labels on top of bars
    for i, (count, percentage) in enumerate(zip(room_counts.values, percentages)):
        plt.text(i, count + 0.1, f'{count} ({percentage:.1f}%)', ha='center', fontweight='bold')
    
    plt.title('Room Type Distribution', fontsize=16, fontweight='bold')
    plt.ylabel('Number of Bookings', fontsize=12)
    plt.tight_layout()
    
    # Add a legend showing room colors
    legend_elements = [plt.Rectangle((0, 0), 1, 1, color=color, label=room) 
                       for room, color in room_colors.items() if room in room_counts.index]
    plt.legend(handles=legend_elements, loc='best')
    
    plt.savefig('room_distribution_consistent.png', dpi=300)
    plt.close()
    print("Room distribution visualization with consistent colors saved as 'room_distribution_consistent.png'")

def standardize_room_name(room_name):
    """
    Standardize room names to ensure consistent matching regardless of 
    minor differences in spacing or capitalization.
    """
    if not isinstance(room_name, str) or pd.isna(room_name):
        return None
    
    # Strip whitespace and convert to lowercase for comparison
    room_name = room_name.strip().lower()
    
    # Define mappings for standard room names
    standard_rooms = {
        'léon de la cour': 'Léon de la Cour',
        'leon de la cour': 'Léon de la Cour',
        'laurasie de la cour': 'Laurasie de la Cour',
        'tibert de la cour': 'Tibert de la Cour',
        'odette de la cour': 'Odette de la Cour'
    }
    
    # Check if this room name matches any of our standard rooms
    for pattern, standard_name in standard_rooms.items():
        if room_name == pattern or room_name.startswith(pattern):
            return standard_name
    
    # If no match found, return the original but with proper capitalization and no extra spaces
    return ' '.join(word.capitalize() for word in room_name.split())

def create_booking_visualizations(df, weekly_booking_goal=GOAL):
    """Create enhanced visualizations of booking data as separate files"""
    # Set style
    sns.set(style="whitegrid")
    
    # Convert dates to datetime format if necessary
    if 'booking_date' in df.columns:
        df['booking_date'] = pd.to_datetime(df['booking_date'], format='%d/%m/%Y', errors='coerce')
    
    if 'arrival_date' in df.columns:
        df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%d/%m/%Y', errors='coerce')
    
    if 'departure_date' in df.columns:
        df['departure_date'] = pd.to_datetime(df['departure_date'], format='%d/%m/%Y', errors='coerce')
    
    # Plot 1: Bookings over time with weekly goal line
    if 'booking_date' in df.columns:
        df_valid_dates = df.dropna(subset=['booking_date']).copy()
        # Exclude Manual bookings and Cancelled bookings
        df_valid_dates = df_valid_dates[(df_valid_dates['booking_source'] != 'Manual') & 
                                       (df_valid_dates['status'] != 'Cancelled')]
        if not df_valid_dates.empty:
            plt.figure(figsize=(10, 6))
            
            # Add week_start column using direct weekday calculation
            df_valid_dates.loc[:, 'week_start'] = df_valid_dates['booking_date'].apply(
                lambda x: x - pd.Timedelta(days=x.weekday())
            )
            
            # Group by week start and count
            bookings_by_week = df_valid_dates.groupby('week_start').size()
            
            # Create custom x-tick labels
            date_labels = []
            for week_start in bookings_by_week.index:
                week_end = week_start + pd.Timedelta(days=6)
                date_labels.append(f"{week_start.strftime('%d/%m/%Y')} - {week_end.strftime('%d/%m/%Y')}")
            
            # Plot bookings by week
            bars = plt.bar(range(len(bookings_by_week)), bookings_by_week.values, color='blue')
            
            # Add horizontal goal line
            plt.axhline(y=weekly_booking_goal, color='red', linestyle='--', label=f'Goal: {weekly_booking_goal} bookings/week')
            
            # Color bars based on goal achievement
            for i, v in enumerate(bookings_by_week.values):
                bars[i].set_color('green' if v >= weekly_booking_goal else 'orange')
            
            # Add trend line with polynomial regression
            x = np.array(range(len(bookings_by_week)))
            y = bookings_by_week.values
            
            # Fit polynomial regression (degree=2 for quadratic fit - adjust as needed)
            z = np.polyfit(x, y, 2)
            p = np.poly1d(z)
            
            # Generate values for the trend line
            trend_x = np.linspace(0, len(bookings_by_week) - 1, 100)
            trend_y = p(trend_x)
            
            # Plot trend line
            plt.plot(trend_x, trend_y, 'b--', linewidth=2, label='Booking Trend')
            
            # Set x-axis labels
            plt.xticks(range(len(bookings_by_week)), date_labels, rotation=45, ha='right')
            plt.xlabel('Week (Monday-Sunday)')
            plt.ylabel('Number of Bookings')
            plt.title('Weekly Booking Count vs Goal')
            plt.legend()
            plt.tight_layout()
            plt.savefig('weekly_booking_count.png')
            plt.close()
            print("Weekly booking count visualization saved as 'weekly_booking_count.png'")
    
    # Plot 2: Booking sources pie chart with nicer colors
    if 'booking_source' in df.columns:
        plt.figure(figsize=(8, 8))
        
        # Set a nicer color palette
        # Using a custom colorblind-friendly palette
        color_palette = sns.color_palette("husl", len(df['booking_source'].unique()))
        
        # Create the pie chart with improved colors
        booking_sources = df['booking_source'].value_counts()
        plt.pie(booking_sources, 
                labels=booking_sources.index, 
                autopct='%1.1f%%', 
                colors=color_palette,
                shadow=False, 
                startangle=90,
                wedgeprops={'edgecolor': 'w', 'linewidth': 1})
        
        plt.title('Booking Sources', fontweight='bold')
        plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
        plt.tight_layout()
        plt.savefig('booking_sources.png')
        plt.close()
        print("Booking sources visualization saved as 'booking_sources.png'")
    
    # Plot 3: Guest arrivals by week
    if 'arrival_date' in df.columns:
        df_valid_arrivals = df.dropna(subset=['arrival_date']).copy()
        if not df_valid_arrivals.empty:
            plt.figure(figsize=(10, 6))
            
            # Add week_start column using direct weekday calculation
            df_valid_arrivals.loc[:, 'week_start'] = df_valid_arrivals['arrival_date'].apply(
                lambda x: x - pd.Timedelta(days=x.weekday())
            )
            
            # Group by week start and count
            arrivals_by_week = df_valid_arrivals.groupby('week_start').size()
            
            # Create custom x-tick labels
            date_labels = []
            for week_start in arrivals_by_week.index:
                week_end = week_start + pd.Timedelta(days=6)
                date_labels.append(f"{week_start.strftime('%d/%m/%Y')} - {week_end.strftime('%d/%m/%Y')}")
            
            # Plot arrivals by week
            arrivals_by_week.plot(kind='bar', color='green')
            plt.title('Guest Arrivals by Week')
            plt.xticks(range(len(date_labels)), date_labels, rotation=45, ha='right')
            plt.xlabel('Week (Monday-Sunday)')
            plt.ylabel('Number of Arrivals')
            plt.tight_layout()
            plt.savefig('guest_arrivals.png')
            plt.close()
            print("Guest arrivals visualization saved as 'guest_arrivals.png'")
    
    # Plot 4: Stay duration histogram
    if 'nights' in df.columns and df['nights'].notna().any():
        plt.figure(figsize=(10, 6))
        sns.histplot(df['nights'].dropna(), kde=True)
        plt.title('Stay Duration Distribution')
        plt.xlabel('Number of Nights')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.savefig('stay_duration.png')
        plt.close()
        print("Stay duration visualization saved as 'stay_duration.png'")
    
    # Plot 5: Room type distribution
    if 'room_type' in df.columns:
        plt.figure(figsize=(10, 6))
        room_counts = df['room_type'].value_counts()
        room_counts.plot(kind='bar', color='purple')
        plt.title('Room Type Distribution')
        plt.ylabel('Number of Bookings')
        plt.xlabel('Room Type')
        
        # Add percentages on top of bars
        total = room_counts.sum()
        for i, v in enumerate(room_counts):
            plt.text(i, v + 0.1, f'{v/total:.1%}', ha='center')
        
        plt.tight_layout()
        plt.savefig('room_distribution.png')
        plt.close()
        print("Room distribution visualization saved as 'room_distribution.png'")
    
    # Plot 6: Enhanced monthly revenue trend with source-dependent payment timing
    if 'amount' in df.columns and 'booking_date' in df.columns and 'departure_date' in df.columns and 'booking_source' in df.columns:
        try:
            # Get current date
            current_date = pd.Timestamp.now().normalize()

            plt.figure(figsize=(12, 8))

            # Clean and convert amount values
            df['amount_numeric'] = df['amount'].astype(str).str.replace(',', '.').replace('', '0')
            df['amount_numeric'] = pd.to_numeric(df['amount_numeric'], errors='coerce')

            # Create a copy of the dataframe for working with
            df_working = df.copy()

            # When filtering for booking sources, also make sure we're excluding Manual from the booking date analyses
            df_booked_to_date = df_working[(df_working['booking_date'] <= current_date) & 
                              (df_working['booking_source'] != 'Manual')]

            # Calculate total confirmed bookings for verification
            total_confirmed = df_working['amount_numeric'].sum()
            print(f"Total Revenue (Confirmed Bookings): €{total_confirmed:.2f}")

            # Split data based on booking source
            df_booking_com = df_working[df_working['booking_source'] == 'Booking.com'].copy()
            df_direct = df_working[df_working['booking_source'].isin(['Website', 'Manual'])].copy()

            # For Booking.com: Revenue is realized on departure date
            # Filter to only include departures up to current date
            df_booking_com_realized = df_booking_com[df_booking_com['departure_date'] <= current_date]

            # For direct bookings: Revenue is realized on booking date
            # Filter to only include bookings up to current date
            df_direct_realized = df_direct[df_direct['booking_date'] <= current_date]

            # Combine the realized revenue dataframes
            df_realized = pd.concat([df_booking_com_realized, df_direct_realized])

            # Calculate total realized revenue for verification
            total_realized = df_realized['amount_numeric'].sum()
            print(f"Total Realized Revenue: €{total_realized:.2f}")

            # Group all bookings by booking month (for booked revenue)
            # Only include bookings up to current date
            df_booked_to_date = df_working[df_working['booking_date'] <= current_date]
            monthly_booked_revenue = df_booked_to_date.groupby(df_booked_to_date['booking_date'].dt.to_period('M'))['amount_numeric'].sum()

            # Group realized revenue by month based on when payment was actually received
            df_realized['revenue_month'] = pd.NaT  # Initialize with NaT (Not a Time)

            # For Booking.com and Website entries, use departure_date as revenue date
            mask_booking_website = df_realized['booking_source'].isin(['Booking.com', 'Website'])
            df_realized.loc[mask_booking_website, 'revenue_month'] = df_realized.loc[mask_booking_website, 'departure_date']

            # For Manual entries only, use booking_date as revenue date
            mask_manual = df_realized['booking_source'] == 'Manual'
            df_realized.loc[mask_manual, 'revenue_month'] = df_realized.loc[mask_manual, 'booking_date']

            # Convert to period for grouping
            df_realized['revenue_month'] = df_realized['revenue_month'].dt.to_period('M')

            # Group by revenue month
            monthly_actual_revenue = df_realized.groupby('revenue_month')['amount_numeric'].sum()

            # Create a unified date range for both series
            all_months = sorted(set(monthly_booked_revenue.index) | set(monthly_actual_revenue.index))

            # Reindex both series to have the same date range, filling missing values with 0
            monthly_booked_revenue = monthly_booked_revenue.reindex(all_months, fill_value=0)
            monthly_actual_revenue = monthly_actual_revenue.reindex(all_months, fill_value=0)

            # Calculate cumulative revenue for both
            cumulative_booked = monthly_booked_revenue.cumsum()
            cumulative_actual = monthly_actual_revenue.cumsum()

            # Verify final totals
            final_booked = cumulative_booked.iloc[-1] if not cumulative_booked.empty else 0
            final_actual = cumulative_actual.iloc[-1] if not cumulative_actual.empty else 0
            print(f"Final Cumulative Booked: €{final_booked:.2f}")
            print(f"Final Cumulative Realized: €{final_actual:.2f}")

            # Setup figure with shared x-axis and two y-axes
            fig, ax1 = plt.subplots(figsize=(14, 8))
            ax2 = ax1.twinx()

            # Plot monthly booked revenue on first axis
            x = range(len(all_months))
            width = 0.35  # width of the bars
            booked_bars = ax1.bar([i - width/2 for i in x], monthly_booked_revenue.values, 
                      width=width, color='#ADD8E6', alpha=0.7, label='Revenue by Booking Date')

            # Plot monthly actual revenue on first axis
            actual_bars = ax1.bar([i + width/2 for i in x], monthly_actual_revenue.values, 
                      width=width, color='#90EE90', alpha=0.7, label='Actual Received Revenue')

            ax1.set_ylabel('Monthly Revenue (€)', fontsize=12)

            # Add value labels on top of bars (only for non-zero values)
            max_val = max(monthly_booked_revenue.max(), monthly_actual_revenue.max())
            label_offset = max_val * 0.02 if max_val > 0 else 20

            for i, v in enumerate(monthly_booked_revenue.values):
                if v > 0:
                    ax1.text(i - width/2, v + label_offset, f'€{v:.0f}', 
                           ha='center', color='blue', fontsize=8, fontweight='bold')

            for i, v in enumerate(monthly_actual_revenue.values):
                if v > 0:
                    ax1.text(i + width/2, v + label_offset, f'€{v:.0f}', 
                           ha='center', color='green', fontsize=8, fontweight='bold')

            # Plot cumulative revenues on second axis
            ax2.plot(x, cumulative_booked.values, 'b-', marker='o', linewidth=2, 
                   markersize=5, label='Cumulative Booked Revenue')
            ax2.plot(x, cumulative_actual.values, 'g-', marker='s', linewidth=2, 
                   markersize=5, label='Cumulative Received Revenue')

            ax2.set_ylabel('Cumulative Revenue (€)', fontsize=12)

            # Format x-axis
            plt.xticks(x, [str(idx).capitalize() for idx in all_months], rotation=45, ha='right')

            # Add title and grid
            plt.title(f'Revenue Comparison (Up to {current_date.strftime("%d/%m/%Y")})', fontsize=16)
            ax1.grid(axis='y', linestyle='--', alpha=0.4)

            # Add legend (combining both axes)
            lines1, labels1 = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            plt.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

            # Add text showing total revenues
            fig.text(0.25, 0.01, f'Total Booked Revenue: €{final_booked:.2f}', 
                    ha='center', fontsize=12, fontweight='bold', color='blue')
            fig.text(0.75, 0.01, f'Total Received Revenue: €{final_actual:.2f}', 
                    ha='center', fontsize=12, fontweight='bold', color='green')

            # Add explanation text
            explanation = "Note: 'Actual Received Revenue' counts Website/Manual bookings on booking date\nand Booking.com reservations on departure date"
            fig.text(0.5, 0.05, explanation, ha='center', fontsize=10, style='italic')

            # Add current date to the figure
            fig.text(0.5, 0.96, f'Generated on: {current_date.strftime("%d/%m/%Y")}', 
                    ha='center', fontsize=10, fontweight='normal')

            plt.tight_layout()
            plt.subplots_adjust(bottom=0.15, top=0.9)  # Make room for texts
            plt.savefig('booking_vs_received_revenue.png', dpi=300)
            plt.close()
            print(f"Revenue comparison up to {current_date.strftime('%d/%m/%Y')} saved as 'booking_vs_received_revenue.png'")

        except Exception as e:
            print(f"Error plotting revenue comparison: {e}")

def create_occupancy_calendar(df):
    """Create improved visualization of room occupancy calendar"""
    if 'room_type' not in df.columns or 'arrival_date' not in df.columns or 'departure_date' not in df.columns:
        print("Cannot create occupancy calendar: missing required columns")
        return None
    
    # Make a copy to avoid modifying the original dataframe
    df_calendar = df.copy()
    
    # Standardize room names
    print("Standardizing room names for occupancy calendar...")
    df_calendar['room_type'] = df_calendar['room_type'].apply(standardize_room_name)
    
    # Remove rows with invalid room types
    df_calendar = df_calendar.dropna(subset=['room_type'])
    
    # Convert dates to datetime - ensure proper conversion using the correct format
    df_calendar['arrival_date'] = pd.to_datetime(df_calendar['arrival_date'], format='%d/%m/%Y', errors='coerce')
    df_calendar['departure_date'] = pd.to_datetime(df_calendar['departure_date'], format='%d/%m/%Y', errors='coerce')
    
    # Remove rows with invalid dates
    df_calendar = df_calendar.dropna(subset=['arrival_date', 'departure_date'])
    
    if df_calendar.empty:
        print("Cannot create occupancy calendar: no valid date data available")
        return None
    
    # Debug: Print the date ranges to verify
    print("Debugging date ranges for occupancy:")
    for idx, row in df_calendar.iterrows():
        print(f"Room: {row['room_type']}, Arrival: {row['arrival_date'].strftime('%d/%m/%Y')}, Departure: {row['departure_date'].strftime('%d/%m/%Y')}")
    
    # Get unique room types
    room_types = sorted(df_calendar['room_type'].unique())
    
    # Create date range for all dates in the dataset
    min_date = df_calendar['arrival_date'].min()
    max_date = df_calendar['departure_date'].max()
    
    # Create date range with a buffer of one month
    date_range = pd.date_range(
        start=min_date - pd.Timedelta(days=15),
        end=max_date + pd.Timedelta(days=15)
    )
    
    # Create occupancy calendar
    occupancy_calendar = pd.DataFrame(index=date_range)
    occupancy_calendar.index.name = 'date'
    
    # Fill occupancy for each room type
    for room in room_types:
        occupancy_calendar[room] = False  # Initialize as not occupied
    
    # Fill in the actual bookings
    for _, booking in df_calendar.iterrows():
        # Make sure room_type is a valid string (not NaN)
        if pd.isna(booking['room_type']):
            continue
            
        # Get dates between arrival and departure (exclusive of departure day)
        stay_dates = pd.date_range(booking['arrival_date'], booking['departure_date'] - pd.Timedelta(days=1))
        
        # Debug: Print the actual stay dates being marked
        print(f"Marking occupancy for {booking['room_type']}: {booking['arrival_date'].strftime('%d/%m/%Y')} to {(booking['departure_date'] - pd.Timedelta(days=1)).strftime('%d/%m/%Y')}")
        
        # Mark these dates as occupied for this room
        for date in stay_dates:
            occupancy_calendar.loc[date, booking['room_type']] = True
    
    # Process calendar by month
    all_months = sorted(set([(d.year, d.month) for d in occupancy_calendar.index]))
    
    for year, month in all_months:
        # Filter data for this month
        month_start = pd.Timestamp(year=year, month=month, day=1)
        if month == 12:
            month_end = pd.Timestamp(year=year+1, month=1, day=1) - pd.Timedelta(days=1)
        else:
            month_end = pd.Timestamp(year=year, month=month+1, day=1) - pd.Timedelta(days=1)
            
        month_data = occupancy_calendar[(occupancy_calendar.index >= month_start) & 
                                       (occupancy_calendar.index <= month_end)]
        
        # Create a better visual calendar for this month
        create_monthly_calendar(month_data, year, month, room_types)

def create_monthly_calendar(month_data, year, month, room_types):
    """Create a visually appealing monthly calendar with fixed colors for rooms and departure indicators"""
    month_name = pd.Timestamp(year=year, month=month, day=1).strftime('%B %Y')
    
    # Determine the number of days in the month and the day of week the month starts
    first_day = pd.Timestamp(year=year, month=month, day=1)
    if month == 12:
        last_day = pd.Timestamp(year=year+1, month=1, day=1) - pd.Timedelta(days=1)
    else:
        last_day = pd.Timestamp(year=year, month=month+1, day=1) - pd.Timedelta(days=1)
    
    days_in_month = last_day.day
    weekday_start = first_day.weekday()  # Monday is 0, Sunday is 6
    
    # Create a figure for this month
    fig, ax = plt.subplots(figsize=(15, 10))
    
    # Remove axis
    ax.axis('off')
    
    # Add month title
    ax.text(0.5, 0.95, month_name, fontsize=24, ha='center', va='top', fontweight='bold')
    
    # Calendar layout parameters
    num_rows = 6  # maximum 6 weeks in a month display
    num_cols = 7  # 7 days in a week
    cell_width = 1.0 / num_cols
    cell_height = 0.85 / num_rows
    
    # Define base colors for each room
    room_colors = {
        'Laurasie de la Cour': 'pink',
        'Tibert de la Cour': '#FFBF00',
        'Odette de la Cour': 'grey',
        'Léon de la Cour': 'teal',  # A blue-green color
        'laurasie': 'pink',
        'tibert': '#FFBF00',
        'odette': 'grey',
        'leon': 'teal',
        'léon': 'teal'  # Adding both with and without accent
    }
    
    # Fallback colors for rooms not in the custom colors dict
    fallback_colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown']
    
    # Day names at the top
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for i, day in enumerate(days):
        ax.text(i * cell_width + cell_width / 2, 0.9, 
                day, ha='center', va='center', fontweight='bold')
    
    # Create a dictionary to track departures
    departures = {}
    for room in room_types:
        departures[room] = []
    
    # Draw calendar grid and fill in the occupancy data
    day_counter = 1
    
    for row in range(num_rows):
        for col in range(num_cols):
            # Calculate cell position
            x = col * cell_width
            y = 0.9 - (row + 1) * cell_height
            
            # Check if this position should have a day number
            if (row == 0 and col < weekday_start) or (day_counter > days_in_month):
                # This cell is outside the current month
                continue
            else:
                # Get current date for this cell
                current_date = pd.Timestamp(year=year, month=month, day=day_counter)
                
                # Draw cell rectangle
                rect = plt.Rectangle((x, y), cell_width, cell_height, 
                                     fill=False, edgecolor='black', linewidth=1)
                ax.add_patch(rect)
                
                # Add day number
                ax.text(x + 0.05, y + cell_height - 0.05, str(day_counter), 
                       ha='left', va='top', fontsize=12)
                
                # Track occupied rooms and departure rooms separately
                occupied_rooms = []
                departure_rooms = []
                
                for room in room_types:
                    # Check if this date is occupied
                    if current_date in month_data.index and month_data.loc[current_date, room]:
                        occupied_rooms.append(room)
                    
                    # Check if this is a departure date (yesterday was occupied but today is not)
                    prev_date = current_date - pd.Timedelta(days=1)
                    if (prev_date in month_data.index and month_data.loc[prev_date, room] and 
                        (current_date not in month_data.index or not month_data.loc[current_date, room])):
                        departure_rooms.append(room)
                
                # Get the width available for indicators
                available_width = cell_width - 0.1
                
                # Calculate spacing for occupied indicators
                if occupied_rooms:
                    occupied_start_y = y + cell_height * 0.6  # Higher position
                    spacing = min(0.1, available_width / len(occupied_rooms))
                    
                    for i, room in enumerate(occupied_rooms):
                        # Set the x position to distribute indicators evenly
                        indicator_x = x + 0.05 + (i * spacing) + (spacing / 2)
                        
                        # Get color for this room
                        color = room_colors.get(room, fallback_colors[i % len(fallback_colors)])
                        
                        # Draw circle for occupied nights
                        circle = plt.Circle(
                            (indicator_x, occupied_start_y),
                            radius=min(0.03, spacing / 2.5),  # Scale circle size
                            color=color,
                            alpha=0.8
                        )
                        ax.add_patch(circle)
                
                # Calculate spacing for departure indicators
                if departure_rooms:
                    departure_start_y = y + cell_height * 0.3  # Lower position
                    spacing = min(0.1, available_width / len(departure_rooms))
                    
                    for i, room in enumerate(departure_rooms):
                        # Set the x position to distribute indicators evenly
                        indicator_x = x + 0.05 + (i * spacing) + (spacing / 2)
                        
                        # Get color for this room
                        color = room_colors.get(room, fallback_colors[i % len(fallback_colors)])
                        
                        # Draw triangle for departure days
                        triangle_size = min(0.04, spacing / 2)
                        triangle = plt.Polygon([
                            [indicator_x, departure_start_y - triangle_size],
                            [indicator_x - triangle_size, departure_start_y + triangle_size],
                            [indicator_x + triangle_size, departure_start_y + triangle_size]
                        ], closed=True, color=color, alpha=0.8)
                        ax.add_patch(triangle)
                
                day_counter += 1
    
    # Add room legend with both symbols
    plt.subplots_adjust(bottom=0.1)
    legend_elements = []
    
    # First add the regular occupancy indicators
    for i, room in enumerate(room_types):
        # Get color for this room
        color = room_colors.get(room, fallback_colors[i % len(fallback_colors)])
        
        # Add circle for "staying"
        legend_elements.append(
            plt.Line2D([0], [0], marker='o', color='w', 
                       markerfacecolor=color, 
                       markersize=10, label=f"{room} (occupied)")
        )
        
        # Add triangle for "departure"
        legend_elements.append(
            plt.Line2D([0], [0], marker='^', color='w', 
                       markerfacecolor=color, 
                       markersize=10, label=f"{room} (checkout)")
        )
    
    ax.legend(handles=legend_elements, loc='upper center', 
              bbox_to_anchor=(0.5, 0.03), ncol=min(4, len(room_types)))
    
    plt.tight_layout()
    plt.savefig(f'occupancy_calendar_{year}_{month:02d}.png', dpi=300)
    plt.close()
    print(f"Occupancy calendar for {month_name} saved as 'occupancy_calendar_{year}_{month:02d}.png'")
        
def analyze_repeat_guests(df):
    """Analyze repeat guests and their booking patterns"""
    if df.empty:
        return "No data available for repeat guest analysis"
    
    results = {}
    
    # Check if we have guest identification columns
    has_email = 'email' in df.columns and df['email'].notna().any()
    has_name = 'guest_name' in df.columns and df['guest_name'].notna().any()
    
    if not (has_email or has_name):
        return "No guest identification columns (email or name) available"
    
    # Prefer email for identification as it's more unique
    id_column = 'email' if has_email else 'guest_name'
    
    # Get repeat guests
    guest_counts = df[id_column].value_counts()
    repeat_guests = guest_counts[guest_counts > 1]
    
    if len(repeat_guests) == 0:
        return "No repeat guests found in the data"
    
    results['repeat_guest_count'] = len(repeat_guests)
    results['repeat_booking_count'] = sum(repeat_guests) - len(repeat_guests)
    
    # For each repeat guest, analyze their booking pattern
    guest_details = []
    
    for guest, booking_count in repeat_guests.items():
        guest_bookings = df[df[id_column] == guest].copy()
        
        # Convert dates to datetime for analysis
        if 'booking_date' in guest_bookings.columns:
            guest_bookings['booking_date'] = pd.to_datetime(guest_bookings['booking_date'], 
                                                            format='%d/%m/%Y', errors='coerce')
        if 'arrival_date' in guest_bookings.columns:
            guest_bookings['arrival_date'] = pd.to_datetime(guest_bookings['arrival_date'], 
                                                            format='%d/%m/%Y', errors='coerce')
        
        # Sort by booking date
        if 'booking_date' in guest_bookings.columns:
            guest_bookings = guest_bookings.sort_values('booking_date')
        
        # Calculate interval between bookings
        if 'booking_date' in guest_bookings.columns and len(guest_bookings) > 1:
            guest_bookings['prev_booking_date'] = guest_bookings['booking_date'].shift(1)
            guest_bookings['days_since_last_booking'] = (guest_bookings['booking_date'] - 
                                                         guest_bookings['prev_booking_date']).dt.days
            
            avg_interval = guest_bookings['days_since_last_booking'].dropna().mean()
        else:
            avg_interval = None
        
        # Get preferred room types if available
        if 'room_type' in guest_bookings.columns:
            preferred_rooms = guest_bookings['room_type'].value_counts().to_dict()
        else:
            preferred_rooms = {}
        
        # Calculate total revenue from this guest
        if 'amount' in guest_bookings.columns:
            # Clean and convert amount values
            amounts = guest_bookings['amount'].astype(str).str.replace(',', '.').replace('', '0')
            total_spent = pd.to_numeric(amounts, errors='coerce').sum()
        else:
            total_spent = None
        
        # Guest booking details
        guest_detail = {
            'guest_id': guest,
            'booking_count': booking_count,
            'avg_days_between_bookings': avg_interval,
            'preferred_rooms': preferred_rooms,
            'total_spent': total_spent,
            'last_booking_date': guest_bookings['booking_date'].max() if 'booking_date' in guest_bookings.columns else None
        }
        
        guest_details.append(guest_detail)
    
    results['guest_details'] = guest_details
    
    return results

def add_manual_booking(excel_file='lafermedelacour_bookings.xlsx'):
    """
    Function to add a manual booking entry to the bookings spreadsheet.
    Creates a new file if it doesn't exist.
    """
    today = datetime.now().strftime('%d/%m/%Y')
    
    # Create empty dataframe with required columns if file doesn't exist
    if not os.path.exists(excel_file):
        columns = ['booking_source', 'email_received_date', 'booking_date', 'email_type', 'status',
                  'reference', 'room_type', 'arrival_date', 'departure_date', 'amount', 
                  'guest_name', 'phone', 'email', 'nights', 'cancellation_date', 
                  'modification_date', 'notes', 'repeat_guest', 'visit_count']
        df = pd.DataFrame(columns=columns)
    else:
        # Load existing data
        df = pd.read_excel(excel_file)
    
    # Get a new unique reference number
    last_manual_ref = 1
    if 'reference' in df.columns:
        # Look for manual references (numbered 1, 2, 3, etc.)
        manual_refs = [int(ref) for ref in df['reference'] if isinstance(ref, (int, float)) or (isinstance(ref, str) and ref.isdigit())]
        if manual_refs:
            last_manual_ref = max(manual_refs) + 1
    
    # Get booking information from user
    print("\n=== Add Manual Booking ===")
    
    # Room selection
    room_options = ['Laurasie de la Cour', 'Tibert de la Cour', 'Léon de la Cour', 'Odette de la Cour']
    print("\nAvailable Rooms:")
    for i, room in enumerate(room_options, 1):
        print(f"{i}. {room}")

    room_choice = None
    while room_choice is None:
        try:
            choice = int(input("\nSelect room (1-4): "))
            if 1 <= choice <= 4:
                room_choice = room_options[choice-1]
            else:
                print("Please select a valid option (1-4)")
        except ValueError:
            print("Please enter a number")
    
    # Get dates
    arrival_date = input("Arrival date (DD/MM/YYYY): ")
    departure_date = input("Departure date (DD/MM/YYYY): ")
    
    # Calculate nights
    nights = None
    try:
        arrival = datetime.strptime(arrival_date, '%d/%m/%Y')
        departure = datetime.strptime(departure_date, '%d/%m/%Y')
        nights = (departure - arrival).days
        print(f"Stay duration: {nights} nights")
    except:
        print("Date format error - please use DD/MM/YYYY")
    
    # Get guest information
    guest_name = input("Guest name: ")
    phone = input("Phone number: ")
    email = input("Email address: ")
    amount = input("Amount (euros): ").replace(',', '.')
    notes = input("Additional notes: ")

    # Create booking entry
    booking = {
        'booking_source': 'Manual',
        'email_received_date': today,
        'booking_date': today,
        'email_type': 'Booking',
        'status': 'Confirmed',
        'reference': last_manual_ref,
        'room_type': room_choice,
        'arrival_date': arrival_date,
        'departure_date': departure_date,
        'amount': amount,
        'guest_name': guest_name,
        'phone': phone,
        'email': email,
        'nights': nights,
        'notes': notes,
        'repeat_guest': False,
        'visit_count': 1
    }
    
    # Add to dataframe
    df = pd.concat([df, pd.DataFrame([booking])], ignore_index=True)
    
    # Sort by received date in ascending order (oldest first)
    if 'email_received_date' in df.columns:
        df['email_received_date'] = pd.to_datetime(df['email_received_date'], format='%d/%m/%Y', errors='coerce')
        df = df.sort_values('email_received_date', ascending=True)
        df['email_received_date'] = df['email_received_date'].dt.strftime('%d/%m/%Y')
    
    # Check for repeat guests after adding
    if 'guest_name' in df.columns and 'email' in df.columns and 'phone' in df.columns:
        try:
            from identify_repeat_guests import identify_repeat_guests
            df = identify_repeat_guests(df)
        except:
            print("Could not process repeat guest information.")
    
    
    # Ensure proper types for all columns to prevent integer conversion problems
    for col in df.columns:
        if col in ['nights', 'visit_count']:
            # Convert to float first to handle NaN values
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif col == 'amount':
            # Make sure amount is always stored as float with proper decimal point
            df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Save to Excel
    df.to_excel(excel_file, index=False)
    print(f"\n✅ Manual booking added successfully! Reference: {last_manual_ref}")
    print(f"Total bookings now: {len(df)}")
    
    return df

def create_booking_calendar(df, year, month):
    """Create a calendar view showing when bookings were made with different colors by source, heatmap, and summary statistics"""
    try:
        # Filter data for this month
        month_start = pd.Timestamp(year=year, month=month, day=1)
        if month == 12:
            month_end = pd.Timestamp(year=year+1, month=1, day=1) - pd.Timedelta(days=1)
        else:
            month_end = pd.Timestamp(year=year, month=month+1, day=1) - pd.Timedelta(days=1)
            
        month_df = df[(df['booking_date'] >= month_start) & (df['booking_date'] <= month_end)]
        month_df = month_df[(month_df['booking_source'] != 'Manual') & 
                   (month_df['status'] != 'Cancelled')]
        
        # Skip if no bookings in this month
        if month_df.empty:
            return
        
        # Create a count of bookings per day by source
        booking_counts = month_df.groupby([month_df['booking_date'].dt.date, 'booking_source']).size().unstack(fill_value=0)
        
        # Make sure 'Website' and 'Booking.com' columns exist
        if 'Website' not in booking_counts.columns:
            booking_counts['Website'] = 0
        if 'Booking.com' not in booking_counts.columns:
            booking_counts['Booking.com'] = 0
            
        # Add a total column
        booking_counts['Total'] = booking_counts.sum(axis=1)
        
        # Create a total booking count per day (for the heatmap)
        total_booking_counts = month_df.groupby(month_df['booking_date'].dt.date).size()
        
        # Calculate monthly stats
        monthly_total = booking_counts['Total'].sum()
        monthly_website = booking_counts['Website'].sum()
        monthly_booking_com = booking_counts['Booking.com'].sum()
        
        # Calendar layout
        month_name = month_start.strftime('%B %Y')
        
        # Determine the number of days in the month and the day of week the month starts
        days_in_month = month_end.day
        weekday_start = month_start.weekday()  # Monday is 0, Sunday is 6
        
        # Create a figure for this month
        fig, ax = plt.subplots(figsize=(14, 10))  # Increased figure size for more space
        
        # Remove axis
        ax.axis('off')
        
        # Add month title with more space
        ax.text(0.5, 0.98, f"Booking Calendar: {month_name}", fontsize=24, ha='center', va='top', fontweight='bold')
        
        # Add monthly stats below the title
        ax.text(0.5, 0.93, f"Monthly Total: {monthly_total} bookings ({monthly_website} Website, {monthly_booking_com} Booking.com)", 
                fontsize=14, ha='center', va='top')
        
        # Calendar layout parameters
        num_rows = 6  # maximum 6 weeks in a month display
        num_cols = 7  # 7 days in a week
        cell_width = 0.9 / num_cols  # slightly reduced to leave space for weekly stats
        cell_height = 0.8 / num_rows  # slightly reduced from 0.85 to add more space at top
        
        # Starting position for the calendar grid
        cal_start_x = 0.05
        cal_start_y = 0.85  # Moved down to give more space from title
        
        # Day names at the top
        days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        for i, day in enumerate(days):
            ax.text(cal_start_x + i * cell_width + cell_width / 2, cal_start_y, 
                    day, ha='center', va='center', fontweight='bold')
        
        # Source colors
        source_colors = {
            'Website': 'green',
            'Booking.com': 'blue'
        }
        
        # Create a colormap for the heatmap effect
        max_bookings = total_booking_counts.max() if not total_booking_counts.empty else 1
        cmap = plt.cm.YlOrRd
        
        # Track weekly statistics
        weekly_stats = []
        current_week = []
        week_number = 1
        
        # Draw calendar grid
        day_counter = 1
        for row in range(num_rows):
            # Reset weekly counts for each row
            weekly_website = 0
            weekly_booking_com = 0
            weekly_total = 0
            has_days_this_week = False
            
            for col in range(num_cols):
                # Calculate cell position
                x = cal_start_x + col * cell_width
                y = cal_start_y - (row + 1) * cell_height
                
                # Check if this position should have a day number
                if (row == 0 and col < weekday_start) or (day_counter > days_in_month):
                    # This cell is outside the current month
                    continue
                else:
                    has_days_this_week = True
                    # Get current date for this cell
                    current_date = pd.Timestamp(year=year, month=month, day=day_counter).date()
                    
                    # Check if there were bookings on this date
                    num_bookings = total_booking_counts.get(current_date, 0)
                    
                    # Get booking counts for this date by source (if any)
                    website_bookings = booking_counts.loc[current_date, 'Website'] if current_date in booking_counts.index else 0
                    booking_com_bookings = booking_counts.loc[current_date, 'Booking.com'] if current_date in booking_counts.index else 0
                    
                    # Update weekly counts
                    weekly_website += website_bookings
                    weekly_booking_com += booking_com_bookings
                    weekly_total += num_bookings
                    
                    # Set cell color based on number of bookings (heatmap)
                    if num_bookings > 0:
                        color_intensity = min(num_bookings / max_bookings, 1.0) * 0.8
                        cell_color = cmap(color_intensity)
                        
                        # Create rectangle with color intensity based on total booking count
                        rect = plt.Rectangle((x, y), cell_width, cell_height, 
                                             fill=True, edgecolor='black', linewidth=1,
                                             facecolor=cell_color, alpha=0.8)
                    else:
                        # Empty cell
                        rect = plt.Rectangle((x, y), cell_width, cell_height, 
                                             fill=False, edgecolor='black', linewidth=1)
                    
                    ax.add_patch(rect)
                    
                    # Add day number
                    ax.text(x + 0.05, y + cell_height - 0.05, str(day_counter), 
                           ha='left', va='top', fontsize=12)
                    
                    # Add booking counts by source if greater than 0
                    text_y_pos = y + cell_height/2
                    
                    if website_bookings > 0:
                        # Website bookings (green)
                        ax.text(x + cell_width/2 - 0.05, text_y_pos, 
                               f"{website_bookings}", 
                               ha='right', va='center', fontsize=14, fontweight='bold',
                               color=source_colors['Website'])
                    
                    if booking_com_bookings > 0:
                        # Booking.com bookings (blue)
                        ax.text(x + cell_width/2 + 0.05, text_y_pos, 
                               f"{booking_com_bookings}", 
                               ha='left', va='center', fontsize=14, fontweight='bold',
                               color=source_colors['Booking.com'])
                    
                    day_counter += 1
            
            # Add weekly stats to the right of each week if there were any days in this week
            if has_days_this_week:
                week_x = cal_start_x + 7 * cell_width + 0.02  # Just to the right of Sunday
                week_y = cal_start_y - (row + 0.5) * cell_height  # Middle of the week row
                
                # Just add the text without a box
                ax.text(week_x, week_y, 
                       f"Week {week_number}: {weekly_total} total ({weekly_website} W, {weekly_booking_com} B)", 
                       ha='left', va='center', fontsize=10, fontweight='bold')
                
                week_number += 1
        
        # Add a colorbar legend for the heatmap
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0, max_bookings))
        sm.set_array([])
        cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.05, shrink=0.6)
        cbar.set_label('Total Number of Bookings', fontsize=12)
        
        # Add legend for booking sources
        legend_x = 0.5
        legend_y = 0.03
        
        # Website legend (green)
        ax.text(legend_x - 0.18, legend_y, "Website:", ha='right', va='center', fontsize=12)
        ax.text(legend_x - 0.15, legend_y, "●", ha='center', va='center', fontsize=20, color=source_colors['Website'])
        
        # Booking.com legend (blue)
        ax.text(legend_x + 0.12, legend_y, "Booking.com:", ha='right', va='center', fontsize=12)
        ax.text(legend_x + 0.15, legend_y, "●", ha='center', va='center', fontsize=20, color=source_colors['Booking.com'])
        
        plt.tight_layout()
        plt.savefig(f'booking_calendar_{year}_{month:02d}.png', dpi=300)
        plt.close()
        print(f"Booking calendar for {month_name} saved as 'booking_calendar_{year}_{month:02d}.png'")
    except Exception as e:
        print(f"Error creating calendar for {month}/{year}: {e}")
    
def create_additional_visualizations(df):
    """
    Create additional visualizations:
    - Revenue per booking source
    - Average stay price 
    - Day of week booking patterns
    - Calendar view of booking dates
    """
    # Make a copy to avoid changing the original
    df = df.copy()
    
    # Ensure proper date conversions
    if 'booking_date' in df.columns:
        df['booking_date'] = pd.to_datetime(df['booking_date'], format='%d/%m/%Y', errors='coerce')
    
    # Convert amount to numeric, handling different formats and missing values
    if 'amount' in df.columns:
        df['amount_numeric'] = df['amount'].astype(str).str.replace(',', '.').replace('', '0')
        df['amount_numeric'] = pd.to_numeric(df['amount_numeric'], errors='coerce')
        # Fill any NaN values with 0
        df['amount_numeric'] = df['amount_numeric'].fillna(0)
    
    # 1. Revenue per booking source visualization
    try:
        if 'booking_source' in df.columns and 'amount_numeric' in df.columns:
            # Filter confirmed bookings
            booking_df = df[df['status'] == 'Confirmed'].copy() if 'status' in df.columns else df.copy()
            
            # Calculate revenue by source
            source_revenue = booking_df.groupby('booking_source')['amount_numeric'].sum().sort_values(ascending=False)
            
            if not source_revenue.empty:
                plt.figure(figsize=(10, 6))
                
                # Create bar chart with custom colors
                ax = source_revenue.plot(kind='bar', color=sns.color_palette("husl", len(source_revenue)))
                
                # Add revenue values on top of bars
                for i, v in enumerate(source_revenue):
                    ax.text(i, v + (source_revenue.max() * 0.02), f'€{v:.2f}', ha='center', fontweight='bold')
                
                plt.title('Total Revenue by Booking Source', fontsize=14, fontweight='bold')
                plt.ylabel('Total Revenue (€)', fontsize=12)
                plt.xlabel('Booking Source', fontsize=12)
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.savefig('revenue_by_source.png')
                plt.close()
                print("Revenue by booking source visualization saved as 'revenue_by_source.png'")
                
                # Also create a pie chart showing revenue share
                plt.figure(figsize=(10, 8))
                plt.pie(source_revenue, labels=source_revenue.index, autopct='%1.1f%%', 
                        colors=sns.color_palette("husl", len(source_revenue)),
                        startangle=90, shadow=False, 
                        wedgeprops={'edgecolor': 'w', 'linewidth': 1})
                centre_circle = plt.Circle((0, 0), 0.4, fc='white')
                fig = plt.gcf()
                fig.gca().add_artist(centre_circle)
                plt.title('Revenue Share by Booking Source', fontsize=16, fontweight='bold')
                plt.tight_layout()
                plt.savefig('revenue_share_pie.png')
                plt.close()
                print("Revenue share pie chart saved as 'revenue_share_pie.png'")
    except Exception as e:
        print(f"Error creating revenue visualizations: {e}")
    
    # 2. Average stay price visualization
    try:
        if 'room_type' in df.columns and 'amount_numeric' in df.columns and 'nights' in df.columns:
            # Filter confirmed bookings with valid data
            price_df = df[(df['status'] == 'Confirmed') if 'status' in df.columns else True].copy()
            price_df = price_df[price_df['amount_numeric'] > 0]
            price_df = price_df[price_df['nights'] > 0]

            # Calculate average price per night for each room
            price_df['price_per_night'] = price_df['amount_numeric'] / price_df['nights']

            if not price_df.empty:
                plt.figure(figsize=(12, 8))  # Increased figure size

                # Calculate average price per night and total revenue by room type
                room_stats = price_df.groupby('room_type').agg({
                    'price_per_night': 'mean',
                    'amount_numeric': 'sum'
                }).sort_values('price_per_night', ascending=False)

                # Get overall average
                overall_avg = price_df['price_per_night'].mean()

                # Create figure with two y-axes
                fig, ax1 = plt.subplots(figsize=(12, 8))
                ax2 = ax1.twinx()

                # Plot average price bars
                x = range(len(room_stats))
                avg_bars = ax1.bar([i - 0.2 for i in x], room_stats['price_per_night'], 
                         width=0.4, color='skyblue', label='Avg Price Per Night')

                # Plot total revenue bars
                total_bars = ax2.bar([i + 0.2 for i in x], room_stats['amount_numeric'], 
                         width=0.4, color='coral', label='Total Revenue')

                # Add horizontal line for overall average
                ax1.axhline(y=overall_avg, color='navy', linestyle='--', 
                           label=f'Overall Avg: €{overall_avg:.2f}')

                # Add average price values on top of average price bars
                for i, v in enumerate(room_stats['price_per_night']):
                    ax1.text(i - 0.2, v + (room_stats['price_per_night'].max() * 0.02), 
                            f'€{v:.2f}', ha='center', fontweight='bold')

                # Add total revenue values on top of total revenue bars
                for i, v in enumerate(room_stats['amount_numeric']):
                    ax2.text(i + 0.2, v + (room_stats['amount_numeric'].max() * 0.02), 
                            f'€{v:.0f}', ha='center', fontweight='bold')

                # Set labels and title
                ax1.set_title('Average Price per Night & Total Revenue by Room Type', fontsize=14, fontweight='bold')
                ax1.set_ylabel('Average Price per Night (€)', fontsize=12)
                ax2.set_ylabel('Total Revenue (€)', fontsize=12)
                ax1.set_xlabel('Room Type', fontsize=12)
                ax1.set_xticks(range(len(room_stats)))
                ax1.set_xticklabels(room_stats.index, rotation=45, ha='right')

                # Combine legends from both axes
                lines1, labels1 = ax1.get_legend_handles_labels()
                lines2, labels2 = ax2.get_legend_handles_labels()
                ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

                plt.tight_layout()
                plt.savefig('price_and_revenue_by_room.png')
                plt.close()
                print("Price and revenue visualization saved as 'price_and_revenue_by_room.png'")
    except Exception as e:
        print(f"Error creating revenue visualizations: {e}")
    
    # 3. Day of week booking patterns
    try:
        if 'booking_date' in df.columns:
            # Filter for confirmed bookings with valid dates
            df_valid_dates = df.dropna(subset=['booking_date']).copy()
            # Exclude Manual bookings and Cancelled bookings
            df_valid_dates = df_valid_dates[(df_valid_dates['booking_source'] != 'Manual') & 
                                       (df_valid_dates['status'] != 'Cancelled')]
            
            if not df_valid_dates.empty:
                # Extract day of week
                df_valid_dates['day_of_week'] = df_valid_dates['booking_date'].dt.day_name()
                
                # Count bookings by day of week
                day_counts = df_valid_dates['day_of_week'].value_counts()
                
                # Sort by day of week (Monday first)
                days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
                day_counts = day_counts.reindex(days_order).fillna(0).astype(int)
                
                # Skip if all zeros
                if day_counts.sum() > 0:
                    # Calculate percentage of total bookings
                    day_percentages = day_counts / day_counts.sum() * 100
                    
                    # Create a figure with two subplots side by side
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
                    
                    # First subplot: Bar chart of counts
                    bars = ax1.bar(day_counts.index, day_counts.values, color=sns.color_palette("husl", 7))
                    for i, v in enumerate(day_counts):
                        ax1.text(i, v + 0.1, str(v), ha='center', fontweight='bold')
                    
                    ax1.set_title('Number of Bookings by Day of Week', fontsize=14, fontweight='bold')
                    ax1.set_ylabel('Number of Bookings', fontsize=12)
                    ax1.set_xlabel('Day of Week', fontsize=12)
                    ax1.set_xticklabels(day_counts.index, rotation=45, ha='right')
                    
                    # Second subplot: Pie chart of percentages
                    ax2.pie(day_percentages, labels=day_percentages.index, autopct='%1.1f%%',
                           colors=sns.color_palette("husl", 7), startangle=90)
                    ax2.set_title('Percentage of Bookings by Day of Week', fontsize=14, fontweight='bold')
                    
                    plt.tight_layout()
                    plt.savefig('bookings_by_day_of_week.png')
                    plt.close()
                    print("Day of week booking patterns saved as 'bookings_by_day_of_week.png'")
                    
                    # Create a detailed analysis with additional stats
                    if 'amount_numeric' in df_valid_dates.columns:
                        plt.figure(figsize=(12, 6))
                        
                        # Group by day of week and calculate stats
                        day_stats = df_valid_dates.groupby('day_of_week').agg({
                            'booking_date': 'count',
                            'amount_numeric': ['sum', 'mean']
                        })
                        
                        # Flatten the column hierarchy
                        day_stats.columns = ['count', 'total_revenue', 'avg_revenue']
                        day_stats = day_stats.reindex(days_order).fillna(0)
                        
                        # Create a multi-index bar chart
                        ax = plt.subplot(111)
                        
                        # Plot count bars
                        x = np.arange(len(days_order))
                        width = 0.25
                        count_bars = ax.bar(x - width, day_stats['count'].astype(int), width, label='Number of Bookings',
                                           color='skyblue')
                        
                        # Add a second y-axis for revenue
                        ax2 = ax.twinx()
                        
                        # Plot average revenue bars
                        avg_bars = ax2.bar(x, day_stats['avg_revenue'], width, label='Average Revenue (€)',
                                          color='orange')
                        
                        # Plot total revenue bars - Handle potential div by zero
                        max_total = day_stats['total_revenue'].max()
                        if max_total > 0:
                            scaled_total = day_stats['total_revenue'] / max_total * day_stats['count'].max()
                        else:
                            scaled_total = day_stats['total_revenue']
                            
                        total_bars = ax.bar(x + width, scaled_total, width, label='Relative Total Revenue',
                                          color='green')
                        
                        # Add labels
                        ax.set_xlabel('Day of Week', fontsize=12)
                        ax.set_ylabel('Number of Bookings', fontsize=12)
                        ax2.set_ylabel('Revenue (€)', fontsize=12)
                        
                        # Set x-ticks and labels
                        ax.set_xticks(x)
                        ax.set_xticklabels(days_order, rotation=45, ha='right')
                        
                        # Combine legends from both axes
                        lines1, labels1 = ax.get_legend_handles_labels()
                        lines2, labels2 = ax2.get_legend_handles_labels()
                        ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
                        
                        # Add annotations
                        for i, v in enumerate(day_stats['count']):
                            ax.text(i - width, v + 0.1, str(int(v)), ha='center', fontsize=9)
                        
                        for i, v in enumerate(day_stats['avg_revenue']):
                            ax2.text(i, v + 0.1, f'€{v:.2f}', ha='center', fontsize=9)
                        
                        for i, v in enumerate(day_stats['total_revenue']):
                            ax.text(i + width, scaled_total[i] + 0.1, f'€{v:.0f}', ha='center', fontsize=9)
                        
                        plt.title('Detailed Analysis of Bookings by Day of Week', fontsize=14, fontweight='bold')
                        plt.tight_layout()
                        plt.savefig('day_of_week_detailed.png')
                        plt.close()
                        print("Detailed day of week analysis saved as 'day_of_week_detailed.png'")
    except Exception as e:
        print(f"Error creating day of week visualizations: {e}")
    
    # 4. Calendar view of booking dates
    try:
        if 'booking_date' in df.columns:
            # Filter for confirmed bookings with valid dates
            df_valid_dates = df.dropna(subset=['booking_date']).copy()
            # Exclude Manual bookings and Cancelled bookings
            df_valid_dates = df_valid_dates[(df_valid_dates['booking_source'] != 'Manual') & 
                                           (df_valid_dates['status'] != 'Cancelled')]
            
            if not df_valid_dates.empty:
                # Get date range
                min_date = df_valid_dates['booking_date'].min()
                max_date = df_valid_dates['booking_date'].max()
                
                # Process by month
                all_months = sorted(set([(d.year, d.month) for d in pd.date_range(min_date, max_date)]))
                
                for year, month in all_months:
                    create_booking_calendar(df_valid_dates, year, month)
    except Exception as e:
        print(f"Error creating booking calendar: {e}")

if __name__ == "__main__":
    try:
        # Run the script to process booking emails with debug mode turned on
        print("Processing booking emails...")
        bookings_df = process_all_booking_emails(debug=False)
        
        if bookings_df is not None and not bookings_df.empty:

            try:
                # Generate and display analytics
                analytics = weekly_booking_analytics(bookings_df)
                print("\nBooking Analytics:")
                print(f"Total Bookings: {analytics['total_bookings']}")
                
                # Display average stay if available
                if isinstance(analytics['average_stay'], (int, float)):
                    print(f"Average Stay: {analytics['average_stay']:.1f} nights")
                else:
                    print(f"Average Stay: {analytics['average_stay']}")
                
                # Display total revenue if available
                if isinstance(analytics['total_revenue'], (int, float)):
                    print(f"Total Revenue: €{analytics['total_revenue']:.2f}")
                else:
                    print(f"Total Revenue: {analytics['total_revenue']}")
                
                # Display weekly booking information if available
                if isinstance(analytics['weekly_bookings'], pd.Series):
                    print("\nWeekly Booking Counts:")
                    print(analytics['weekly_bookings'])
                else:
                    print(f"\nWeekly Booking Counts: {analytics['weekly_bookings']}")
                
                # Display upcoming arrivals if available
                if isinstance(analytics['upcoming_arrivals'], pd.Series):
                    print("\nUpcoming Arrivals by Week:")
                    print(analytics['upcoming_arrivals'])
                else:
                    print(f"\nUpcoming Arrivals by Week: {analytics['upcoming_arrivals']}")
                
                # Create visualizations
                try:
                    create_booking_visualizations(bookings_df)
                    create_additional_visualizations(bookings_df)
                    create_room_distribution_with_consistent_colors(bookings_df)
                    analyze_guest_nationalities(bookings_df)
                except Exception as e:
                    print(f"Could not create visualizations: {e}")
                
            except Exception as e:
                print(f"\nError generating analytics --------: {e}")
                print("Data was successfully extracted and saved to Excel file.")
            
            # Provide instructions for viewing the data
            print("\nTo view all booking data, open the 'lafermedelacour_bookings.xlsx' file which has been saved to your current directory.")
            
        else:
            print("No booking data was found in your Gmail account.")
    
    except Exception as e:
        print(f"An error occurred while running the script: {e}")
        print("Please make sure your credentials.json file is in the correct location and you have granted the necessary permissions.")

Processing booking emails...
Found existing bookings file with 29 records.
Found 4 manual bookings in the Excel file.

Processing emails with term: 'nouvelle réservation'...
✅ Extracted reference: P2504153258
Extracting booking information from email...
Room type: Tibert de la Cour
Arrival date: 19/04/2025
Departure date: 20/04/2025
Amount: 180,00 €
Guest name: Timo Pentner
Phone: +31 6 41108750
Email: timo.pentner@euroctp.eu
Number of nights: 1
----------------------------------------
✅ Extracted reference: P2504140824
Extracting booking information from email...
Room type: Tibert de la Cour
Arrival date: 15/08/2025
Departure date: 17/08/2025
Amount: 324,00 €
Guest name: Cathy Pittevils
Phone: +0498949938
Email: cathypittevils@hotmail.com
Number of nights: 2
----------------------------------------
✅ Extracted reference: U2504133720
Extracting booking information from email...
Room type: Tibert de la Cour
Arrival date: 26/06/2025
Departure date: 27/06/2025
Amount: 209,00 €
Guest name:

✅ Extracted reference: U2502031538
Extracting booking information from email...
Room type: Laurasie de la Cour
Arrival date: 22/03/2025
Departure date: 23/03/2025
Amount: 318,64 €
Guest name: Bénédicte BILOCQ
Phone: +32 484028023
Email: bbiloc.690370@guest.booking.com
Number of nights: 1
----------------------------------------
✅ Extracted reference: U2502022401
Extracting booking information from email...
Room type: Laurasie de la Cour
Arrival date: 08/02/2025
Departure date: 09/02/2025
Amount: 151,04 €
Guest name: Anne MISSOUL
Phone: +32 497609127
Email: amisso.280952@guest.booking.com
Number of nights: 1
----------------------------------------
✅ Extracted reference: U2502010781
Extracting booking information from email...
Room type: Laurasie de la Cour
Arrival date: 01/02/2025
Departure date: 02/02/2025
Amount: 151,04 €
Guest name: Olivier BALBEUR
Phone: +32 621194262
Email: obalbe.759341@guest.booking.com
Number of nights: 1
----------------------------------------
✅ Extracted ref

⛔ No reference found.
Extracting cancellation information from email...
Guest name: Connectez Votre
Cancellation date: 13/02/2025

== CANCELLATION DATA SUMMARY ==
  booking_source: Booking.com
  email_received_date: 13/02/2025
  booking_date: 13/02/2025
  email_type: Cancellation
  status: Cancelled
  guest_name: Connectez Votre
  cancellation_date: 13/02/2025
----------------------------------------

====== EXTRACTION DATA CHECK ======
Cancellation email for reference: UNKNOWN
Extracted data:
  booking_source: Booking.com
  email_received_date: 13/02/2025
  booking_date: 13/02/2025
  email_type: Cancellation
  status: Cancelled
  guest_name: Connectez Votre
  cancellation_date: 13/02/2025
DEBUG: Ensuring cancellation data integrity...

🔍 DEBUG: Cancellation data after recovery:
  booking_source: Booking.com
  email_received_date: 13/02/2025
  booking_date: 13/02/2025
  email_type: Cancellation
  status: Cancelled
  guest_name: Connectez Votre
  cancellation_date: 13/02/2025
⛔ Skipped 


DEBUG: Converting all integer columns to float in manual_df

DEBUG: Column types before concat:
  merged_email_df[booking_source]: object
  merged_email_df[booking_date]: object
  merged_email_df[email_type]: object
  merged_email_df[status]: object
  merged_email_df[reference]: object
  merged_email_df[room_type]: object
  merged_email_df[arrival_date]: object
  merged_email_df[departure_date]: object
  merged_email_df[amount]: float64
  merged_email_df[guest_name]: object
  merged_email_df[phone]: object
  merged_email_df[email]: object
  merged_email_df[nights]: float64
  merged_email_df[cancellation_date]: object
  merged_email_df[modification_date]: object
  merged_email_df[notes]: object
  merged_email_df[repeat_guest]: object
  merged_email_df[visit_count]: float64
  manual_df[booking_source]: object
  manual_df[email_received_date]: object
  manual_df[booking_date]: object
  manual_df[email_type]: object
  manual_df[status]: object
  manual_df[reference]: object
  manual_df[ro

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


...

C:\Users\jules\AppData\Local\Temp/ipykernel_34128/1100303271.py:2910: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(day_counts.index, rotation=45, ha='right')


Day of week booking patterns saved as 'bookings_by_day_of_week.png'
Detailed day of week analysis saved as 'day_of_week_detailed.png'
Booking calendar for January 2025 saved as 'booking_calendar_2025_01.png'
Booking calendar for February 2025 saved as 'booking_calendar_2025_02.png'
Booking calendar for March 2025 saved as 'booking_calendar_2025_03.png'
Booking calendar for April 2025 saved as 'booking_calendar_2025_04.png'
Room distribution visualization with consistent colors saved as 'room_distribution_consistent.png'

Sample phone number to nationality mappings:
Phone: +0498949938 → Nationality: Belgium
Phone: +32 493 85 93 85 → Nationality: Belgium
Phone: 3267636407 → Nationality: Belgium
Phone: +31 6 41108750 → Nationality: Netherlands
Phone: +32 473 54 75 48 → Nationality: Belgium
Phone: 32476633922 → Nationality: Belgium
Phone: +32 621194262 → Nationality: Belgium
Phone: +32 479 80 35 75 → Nationality: Belgium
Phone: +32 470 61 43 13 → Nationality: Belgium
Phone: +31 6 11929809 

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>

## Improvements:

1. check for annulations -> mark them and update. do not erase the contact info in case we want to email them or something. 

1.1. check for modification and update ! replace all info this time. 

2. add in xlsx file if its a guest's second or more stay (if coming back to the guest house)

3. have the xlsx with a columns "notes"so that my mother in law (the owner) can write some notes abouot the guests in case they come back. Be able to open the existing file and pdate it then. 

-> have the xlsx with a columns "notes"so that my mother in law (the owner) can write some notes abouot the guests in case they come back. Be able to open the existing file and update it then. maybe just check with name and arrival date if already processed or something so that I do not duplicate some lines.

4. deal with Airbnb


(+ add nationality from phone number)

airbnb (wrong)

Subject: Réservation confirmée : Xavier Gobbo arrive le 27 mai
From: Airbnb <automated@airbnb.com>

Raw plain text excerpt:
%opentrack%

https://fr.airbnb.be/?c=.pi80.pkYm9va2luZy92Ml9taWdyYXRpb24vcmVzZXJ2YXRpb25faG9zdF9jb25maXJtYXRpb24=&euid=f072a0fa-cc58-374a-868f-09bff9142f90

NOUVELLE RÉSERVATION CONFIRMÉE ! XAVIER ARRIVE LE 27 MAI

Envoyez un message pour confirmer les détails de l'entrée
dans les lieux ou po

Specific content we're looking for:
  Name: NOT FOUND
  Arrival: NOT FOUND
  Departure: NOT FOUND
  Amount: NOT FOUND
----------------------------------------
Extracting cancellation information from email...
Cancellation date: 24/02/2025
----------------------------------------
Extracted information:
  booking_source: Booking.com
  email_received_date: 24/02/2025
  booking_date: 24/02/2025
  email_type: Cancellation
  cancellation_date: 24/02/2025